# Analisi e Previsione della Produzione di Olio d'Oliva

Questo notebook esplora la relazione tra i dati meteorologici e la produzione annuale di olio d'oliva, con l'obiettivo di creare un modello predittivo.

In [ ]:
!apt-get update
!apt-get install graphviz -y

!pip install tensorflow
!pip install numpy
!pip install pandas

!pip install keras
!pip install scikit-learn
!pip install matplotlib
!pip install joblib
!pip install pyarrow
!pip install fastparquet
!pip install scipy
!pip install seaborn
!pip install tqdm
!pip install pydot
!pip install tensorflow-io

In [ ]:
import tensorflow as tf
import keras

print(f"Keras version: {keras.__version__}")
print(f"TensorFlow version: {tf.__version__}")
print(f"TensorFlow version: {tf.__version__}")
print(f"CUDA available: {tf.test.is_built_with_cuda()}")
print(f"GPU devices: {tf.config.list_physical_devices('GPU')}")

# GPU configuration
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

In [ ]:
# Test semplice per verificare che la GPU funzioni
def test_gpu():
    print("TensorFlow version:", tf.__version__)
    print("\nDispositivi disponibili:")
    print(tf.config.list_physical_devices())

    # Creiamo e moltiplichiamo due tensori sulla GPU
    with tf.device('/GPU:0'):
        a = tf.random.normal([10000, 10000])
        b = tf.random.normal([10000, 10000])
        c = tf.matmul(a, b)

    print("\nShape del risultato:", c.shape)
    print("Device del tensore:", c.device)
    return "Test completato con successo!"


test_gpu()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.layers import Input, Dense, Dropout, Bidirectional, LSTM, LayerNormalization, Add, Activation, BatchNormalization, MultiHeadAttention, MaxPooling1D, Conv1D, GlobalMaxPooling1D, GlobalAveragePooling1D, \
    Concatenate, ZeroPadding1D, Lambda, AveragePooling1D, concatenate
from tensorflow.keras.layers import Dense, LSTM, Conv1D, Input, concatenate, Dropout, BatchNormalization, GlobalAveragePooling1D, Bidirectional, TimeDistributed, Attention, MultiHeadAttention
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from datetime import datetime
import os
import json
import joblib
import re
import pyarrow as pa
import pyarrow.parquet as pq
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor, as_completed
from functools import partial
import psutil
import multiprocessing

random_state_value = 42

base_project_dir = './kaggle/working/'
data_project_dir = base_project_dir + 'data/'
models_project_dir = base_project_dir + 'models/'

os.makedirs(base_project_dir, exist_ok=True)
os.makedirs(data_project_dir, exist_ok=True)
os.makedirs(models_project_dir, exist_ok=True)

## Funzioni di Plot

In [ ]:
def save_plot(plt, title, output_dir='./kaggle/working/plots'):
    os.makedirs(output_dir, exist_ok=True)
    filename = "".join(x for x in title if x.isalnum() or x in [' ', '-', '_']).rstrip()
    filename = filename.replace(' ', '_').lower()
    filepath = os.path.join(output_dir, f"{filename}.png")
    plt.savefig(filepath, bbox_inches='tight', dpi=300)
    print(f"Plot salvato come: {filepath}")


def to_camel_case(text):
    """
    Converte una stringa in camelCase.
    Gestisce stringhe con spazi, trattini o underscore.
    Se è una sola parola, la restituisce in minuscolo.
    """
    # Rimuove eventuali spazi iniziali e finali
    text = text.strip()

    # Se la stringa è vuota, ritorna stringa vuota
    if not text:
        return ""

    # Sostituisce trattini e underscore con spazi
    text = text.replace('-', ' ').replace('_', ' ')

    # Divide la stringa in parole
    words = text.split()

    # Se non ci sono parole dopo lo split, ritorna stringa vuota
    if not words:
        return ""

    # Se c'è una sola parola, ritorna in minuscolo
    if len(words) == 1:
        return words[0].lower()

    # Altrimenti procedi con il camelCase
    result = words[0].lower()
    for word in words[1:]:
        result += word.capitalize()

    return result

## 1. Caricamento e preparazione dei Dati Meteo

In [ ]:
# Function to convert csv to parquet
def csv_to_parquet(csv_file, parquet_file, chunksize=100000):
    writer = None

    for chunk in pd.read_csv(csv_file, chunksize=chunksize):
        if writer is None:

            table = pa.Table.from_pandas(chunk)
            writer = pq.ParquetWriter(parquet_file, table.schema)
        else:
            table = pa.Table.from_pandas(chunk)

        writer.write_table(table)

    if writer:
        writer.close()

    print(f"File conversion completed : {csv_file} -> {parquet_file}")


def read_json_files(folder_path):
    all_data = []

    file_list = sorted(os.listdir(folder_path))

    for filename in file_list:
        if filename.endswith('.json'):
            file_path = os.path.join(folder_path, filename)
            try:
                with open(file_path, 'r') as file:
                    data = json.load(file)
                    all_data.extend(data['days'])
            except Exception as e:
                print(f"Error processing file '{filename}': {str(e)}")

    return all_data


def create_weather_dataset(data):
    dataset = []
    seen_datetimes = set()

    for day in data:
        date = day['datetime']
        for hour in day['hours']:
            datetime_str = f"{date} {hour['datetime']}"

            # Verifico se questo datetime è già stato visto
            if datetime_str in seen_datetimes:
                continue

            seen_datetimes.add(datetime_str)

            if isinstance(hour['preciptype'], list):
                preciptype = "__".join(hour['preciptype'])
            else:
                preciptype = hour['preciptype'] if hour['preciptype'] else ""

            conditions = hour['conditions'].replace(', ', '__').replace(' ', '_').lower()

            row = {
                'datetime': datetime_str,
                'temp': hour['temp'],
                'feelslike': hour['feelslike'],
                'humidity': hour['humidity'],
                'dew': hour['dew'],
                'precip': hour['precip'],
                'snow': hour['snow'],
                'preciptype': preciptype.lower(),
                'windspeed': hour['windspeed'],
                'winddir': hour['winddir'],
                'pressure': hour['pressure'],
                'cloudcover': hour['cloudcover'],
                'visibility': hour['visibility'],
                'solarradiation': hour['solarradiation'],
                'solarenergy': hour['solarenergy'],
                'uvindex': hour['uvindex'],
                'conditions': conditions,
                'tempmax': day['tempmax'],
                'tempmin': day['tempmin'],
                'precipprob': day['precipprob'],
                'precipcover': day['precipcover']
            }
            dataset.append(row)

    dataset.sort(key=lambda x: datetime.strptime(x['datetime'], "%Y-%m-%d %H:%M:%S"))

    return pd.DataFrame(dataset)



In [ ]:
# Crea le sequenze per LSTM
def create_sequences(timesteps, X, y=None):
    """
    Crea sequenze temporali dai dati.
    
    Parameters:
    -----------
    X : array-like
        Dati di input
    timesteps : int
        Numero di timestep per ogni sequenza
    y : array-like, optional
        Target values. Se None, crea sequenze solo per X
        
    Returns:
    --------
    tuple o array
        Se y è fornito: (X_sequences, y_sequences)
        Se y è None: X_sequences
    """
    Xs = []
    for i in range(len(X) - timesteps):
        Xs.append(X[i:i + timesteps])

    if y is not None:
        ys = []
        for i in range(len(X) - timesteps):
            ys.append(y[i + timesteps])
        return np.array(Xs), np.array(ys)

    return np.array(Xs)


def get_season(date):
    month = date.month
    day = date.day
    if (month == 12 and day >= 21) or (month <= 3 and day < 20):
        return 'Winter'
    elif (month == 3 and day >= 20) or (month <= 6 and day < 21):
        return 'Spring'
    elif (month == 6 and day >= 21) or (month <= 9 and day < 23):
        return 'Summer'
    elif (month == 9 and day >= 23) or (month <= 12 and day < 21):
        return 'Autumn'
    else:
        return 'Unknown'


def get_time_period(hour):
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'


def add_time_features(df):
    df['datetime'] = pd.to_datetime(df['datetime'])
    df['timestamp'] = df['datetime'].astype(np.int64) // 10 ** 9
    df['year'] = df['datetime'].dt.year
    df['month'] = df['datetime'].dt.month
    df['day'] = df['datetime'].dt.day
    df['hour'] = df['datetime'].dt.hour
    df['minute'] = df['datetime'].dt.minute
    df['hour_sin'] = np.sin(df['hour'] * (2 * np.pi / 24))
    df['hour_cos'] = np.cos(df['hour'] * (2 * np.pi / 24))
    df['day_of_week'] = df['datetime'].dt.dayofweek
    df['day_of_year'] = df['datetime'].dt.dayofyear
    df['week_of_year'] = df['datetime'].dt.isocalendar().week.astype(int)
    df['quarter'] = df['datetime'].dt.quarter
    df['is_month_end'] = df['datetime'].dt.is_month_end.astype(int)
    df['is_quarter_end'] = df['datetime'].dt.is_quarter_end.astype(int)
    df['is_year_end'] = df['datetime'].dt.is_year_end.astype(int)
    df['month_sin'] = np.sin(df['month'] * (2 * np.pi / 12))
    df['month_cos'] = np.cos(df['month'] * (2 * np.pi / 12))
    df['day_of_year_sin'] = np.sin(df['day_of_year'] * (2 * np.pi / 365.25))
    df['day_of_year_cos'] = np.cos(df['day_of_year'] * (2 * np.pi / 365.25))
    df['season'] = df['datetime'].apply(get_season)
    df['time_period'] = df['hour'].apply(get_time_period)
    return df


def add_solar_features(df):
    # Calcolo dell'angolo solare
    df['solar_angle'] = np.sin(df['day_of_year'] * (2 * np.pi / 365.25)) * np.sin(df['hour'] * (2 * np.pi / 24))

    # Interazioni tra features rilevanti
    df['cloud_temp_interaction'] = df['cloudcover'] * df['temp']
    df['visibility_cloud_interaction'] = df['visibility'] * (100 - df['cloudcover'])

    # Feature derivate
    df['clear_sky_index'] = (100 - df['cloudcover']) / 100
    df['temp_gradient'] = df['temp'] - df['tempmin']

    return df


def add_solar_specific_features(df):
    # Angolo solare e durata del giorno
    df['day_length'] = 12 + 3 * np.sin(2 * np.pi * (df['day_of_year'] - 81) / 365.25)
    df['solar_noon'] = 12 - df['hour']
    df['solar_elevation'] = np.sin(2 * np.pi * df['day_of_year'] / 365.25) * np.cos(2 * np.pi * df['solar_noon'] / 24)

    # Interazioni
    df['cloud_elevation'] = df['cloudcover'] * df['solar_elevation']
    df['visibility_elevation'] = df['visibility'] * df['solar_elevation']

    # Rolling features con finestre più ampie
    df['cloud_rolling_12h'] = df['cloudcover'].rolling(window=12).mean()
    df['temp_rolling_12h'] = df['temp'].rolling(window=12).mean()

    return df


def add_advanced_features(df):
    # Features esistenti
    df = add_time_features(df)
    df = add_solar_features(df)
    df = add_solar_specific_features(df)

    # Aggiungi interazioni tra variabili meteorologiche
    df['temp_humidity'] = df['temp'] * df['humidity']
    df['temp_cloudcover'] = df['temp'] * df['cloudcover']
    df['visibility_cloudcover'] = df['visibility'] * df['cloudcover']

    # Features derivate per la radiazione solare
    df['clear_sky_factor'] = (100 - df['cloudcover']) / 100
    df['day_length'] = np.sin(df['day_of_year_sin']) * 12 + 12  # approssimazione della durata del giorno

    # Lag features
    df['temp_1h_lag'] = df['temp'].shift(1)
    df['cloudcover_1h_lag'] = df['cloudcover'].shift(1)
    df['humidity_1h_lag'] = df['humidity'].shift(1)

    # Rolling means
    df['temp_rolling_mean_6h'] = df['temp'].rolling(window=6).mean()
    df['cloudcover_rolling_mean_6h'] = df['cloudcover'].rolling(window=6).mean()

    return df


# Preparazione dati
def prepare_solar_data(weather_data, features):
    """
    Prepara i dati per i modelli solari.
    """
    # Aggiungi le caratteristiche temporali
    weather_data = add_advanced_features(weather_data)
    weather_data = pd.get_dummies(weather_data, columns=['season', 'time_period'], drop_first=True)

    # Dividi i dati
    data_after_2010 = weather_data[weather_data['year'] >= 2010].copy()
    data_after_2010 = data_after_2010.sort_values('datetime')
    data_after_2010.set_index('datetime', inplace=True)

    # Interpola valori mancanti
    target_variables = ['solarradiation', 'solarenergy', 'uvindex']
    for column in target_variables:
        data_after_2010[column] = data_after_2010[column].interpolate(method='time')

    # Rimuovi righe con valori mancanti
    data_after_2010.dropna(subset=features + target_variables, inplace=True)

    # Prepara X e y
    X = data_after_2010[features].values
    y = data_after_2010[target_variables].values

    # Normalizza features
    scaler_X = MinMaxScaler()
    X_scaled = scaler_X.fit_transform(X)

    scaler_y = MinMaxScaler()
    y_scaled = scaler_y.fit_transform(y)

    return X_scaled, scaler_X, y_scaled, scaler_y, data_after_2010


def prepare_model_specific_data(X_scaled, y, target_idx, timesteps):
    """
    Prepara i dati specifici per ciascun modello.
    """
    # Scaler specifico per il target
    scaler_y = MinMaxScaler()
    y_scaled = scaler_y.fit_transform(y[:, target_idx].reshape(-1, 1))

    # Split dei dati
    X_train, X_temp, y_train, y_temp = train_test_split(
        X_scaled, y_scaled, test_size=0.3, shuffle=False
    )
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=0.5, shuffle=False
    )

    # Crea sequenze
    X_train_seq, y_train_seq = create_sequences(timesteps, X_train, y_train)
    X_val_seq, y_val_seq = create_sequences(timesteps, X_val, y_val)
    X_test_seq, y_test_seq = create_sequences(timesteps, X_test, y_test)

    return {
        'train': (X_train_seq, y_train_seq),
        'val': (X_val_seq, y_val_seq),
        'test': (X_test_seq, y_test_seq)
    }, scaler_y


def create_radiation_model(input_shape, solar_params_shape=(3,)):
    """
    Modello per la radiazione solare con vincoli di non-negatività.
    """
    # Input layers
    main_input = Input(shape=input_shape, name='main_input')
    solar_input = Input(shape=solar_params_shape, name='solar_params')

    # Branch CNN
    x1 = Conv1D(32, 3, padding='same')(main_input)
    x1 = BatchNormalization()(x1)
    x1 = Activation('relu')(x1)
    x1 = Conv1D(64, 3, padding='same')(x1)
    x1 = BatchNormalization()(x1)
    x1 = Activation('relu')(x1)
    x1 = GlobalAveragePooling1D()(x1)

    # Branch LSTM
    x2 = Bidirectional(LSTM(64, return_sequences=True))(main_input)
    x2 = Bidirectional(LSTM(32))(x2)
    x2 = BatchNormalization()(x2)

    # Solar parameters processing
    x3 = Dense(32)(solar_input)
    x3 = BatchNormalization()(x3)
    x3 = Activation('relu')(x3)

    # Combine all branches
    x = concatenate([x1, x2, x3])

    # Dense layers with non-negativity constraints
    x = Dense(64, kernel_constraint=tf.keras.constraints.NonNeg())(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.2)(x)

    x = Dense(32, kernel_constraint=tf.keras.constraints.NonNeg())(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Output layer con vincoli di non-negatività
    output = Dense(1,
                   kernel_constraint=tf.keras.constraints.NonNeg(),
                   activation='relu')(x)

    model = Model(inputs=[main_input, solar_input], outputs=output, name="SolarRadiation")
    return model


def create_energy_model(input_shape):
    """
    Modello migliorato per l'energia solare che sfrutta la relazione con la radiazione.
    Include vincoli di non-negatività e migliore gestione delle dipendenze temporali.
    """
    inputs = Input(shape=input_shape)

    # Branch 1: Elaborazione temporale con attention
    # Multi-head attention per catturare relazioni temporali
    x1 = MultiHeadAttention(num_heads=8, key_dim=32)(inputs, inputs)
    x1 = BatchNormalization()(x1)
    x1 = Activation('relu')(x1)

    # Temporal Convolution branch per catturare pattern locali
    x2 = Conv1D(
        filters=64,
        kernel_size=3,
        padding='same',
        kernel_constraint=tf.keras.constraints.NonNeg()
    )(inputs)
    x2 = BatchNormalization()(x2)
    x2 = Activation('relu')(x2)
    x2 = Conv1D(
        filters=32,
        kernel_size=3,
        padding='same',
        kernel_constraint=tf.keras.constraints.NonNeg()
    )(x2)
    x2 = BatchNormalization()(x2)
    x2 = Activation('relu')(x2)

    # LSTM branch per memoria a lungo termine
    x3 = LSTM(64, return_sequences=True)(inputs)
    x3 = LSTM(32, return_sequences=False)(x3)
    x3 = BatchNormalization()(x3)
    x3 = Activation('relu')(x3)

    # Global pooling per ogni branch
    x1 = GlobalAveragePooling1D()(x1)
    x2 = GlobalAveragePooling1D()(x2)

    # Concatena tutti i branch
    x = concatenate([x1, x2, x3])

    # Dense layers con vincoli di non-negatività
    x = Dense(
        128,
        kernel_constraint=tf.keras.constraints.NonNeg(),
        kernel_regularizer=l2(0.01)
    )(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.3)(x)

    x = Dense(
        64,
        kernel_constraint=tf.keras.constraints.NonNeg(),
        kernel_regularizer=l2(0.01)
    )(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.2)(x)

    # Output layer con vincolo di non-negatività
    output = Dense(
        1,
        kernel_constraint=tf.keras.constraints.NonNeg(),
        activation='relu',  # Garantisce output non negativo
        kernel_regularizer=l2(0.01)
    )(x)

    model = Model(inputs=inputs, outputs=output, name="SolarEnergy")
    return model


def create_uv_model(input_shape):
    """
    Modello migliorato per l'indice UV che sfrutta sia radiazione che energia solare.
    Include vincoli di non-negatività e considera le relazioni non lineari tra le variabili.
    """
    inputs = Input(shape=input_shape)

    # CNN branch per pattern locali
    x1 = Conv1D(
        filters=64,
        kernel_size=3,
        padding='same',
        kernel_constraint=tf.keras.constraints.NonNeg()
    )(inputs)
    x1 = BatchNormalization()(x1)
    x1 = Activation('relu')(x1)
    x1 = MaxPooling1D(pool_size=2)(x1)

    x1 = Conv1D(
        filters=32,
        kernel_size=3,
        padding='same',
        kernel_constraint=tf.keras.constraints.NonNeg()
    )(x1)
    x1 = BatchNormalization()(x1)
    x1 = Activation('relu')(x1)
    x1 = GlobalAveragePooling1D()(x1)

    # Attention branch per relazioni complesse
    # Specialmente utile per le relazioni con radiazione ed energia
    x2 = MultiHeadAttention(num_heads=4, key_dim=32)(inputs, inputs)
    x2 = BatchNormalization()(x2)
    x2 = Activation('relu')(x2)
    x2 = GlobalAveragePooling1D()(x2)

    # Dense branch per le feature più recenti
    x3 = GlobalAveragePooling1D()(inputs)
    x3 = Dense(
        64,
        kernel_constraint=tf.keras.constraints.NonNeg(),
        kernel_regularizer=l2(0.01)
    )(x3)
    x3 = BatchNormalization()(x3)
    x3 = Activation('relu')(x3)

    # Fusion dei branch
    x = concatenate([x1, x2, x3])

    # Dense layers con vincoli di non-negatività
    x = Dense(
        128,
        kernel_constraint=tf.keras.constraints.NonNeg(),
        kernel_regularizer=l2(0.01)
    )(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.3)(x)

    x = Dense(
        64,
        kernel_constraint=tf.keras.constraints.NonNeg(),
        kernel_regularizer=l2(0.01)
    )(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.2)(x)

    # Output layer con vincolo di non-negatività
    output = Dense(
        1,
        kernel_constraint=tf.keras.constraints.NonNeg(),
        activation='relu',  # Garantisce output non negativo
        kernel_regularizer=l2(0.01)
    )(x)

    model = Model(inputs=inputs, outputs=output, name="SolarUV")
    return model


class CustomCallback(tf.keras.callbacks.Callback):
    """
    Callback personalizzato per monitorare la non-negatività delle predizioni
    e altre metriche importanti durante il training.
    """

    def __init__(self, validation_data=None):
        super().__init__()
        self.validation_data = validation_data

    def on_epoch_end(self, epoch, logs=None):
        try:
            # Controlla se abbiamo i dati di validazione
            if hasattr(self.model, 'validation_data'):
                val_x = self.model.validation_data[0]
                if isinstance(val_x, list):  # Per il modello della radiazione
                    val_pred = self.model.predict(val_x, verbose=0)
                else:
                    val_pred = self.model.predict(val_x, verbose=0)

                # Verifica non-negatività
                if np.any(val_pred < 0):
                    print("\nWarning: Rilevati valori negativi nelle predizioni")
                    print(f"Min value: {np.min(val_pred)}")

                # Statistiche predizioni
                print(f"\nStatistiche predizioni epoca {epoch}:")
                print(f"Min: {np.min(val_pred):.4f}")
                print(f"Max: {np.max(val_pred):.4f}")
                print(f"Media: {np.mean(val_pred):.4f}")

                # Aggiunge le metriche ai logs
                if logs is not None:
                    logs['val_pred_min'] = np.min(val_pred)
                    logs['val_pred_max'] = np.max(val_pred)
                    logs['val_pred_mean'] = np.mean(val_pred)
        except Exception as e:
            print(f"\nWarning nel CustomCallback: {str(e)}")


def create_callbacks(target):
    """
    Crea le callbacks per il training del modello.
    
    Parameters:
    -----------
    target : str
        Nome del target per cui creare le callbacks
        
    Returns:
    --------
    list : Lista delle callbacks configurate
    """
    # Crea la directory per i checkpoint e i logs
    model_dir = f'./kaggle/working/models/{target}'
    checkpoint_dir = os.path.join(model_dir, 'checkpoints')
    log_dir = os.path.join(model_dir, 'logs')

    os.makedirs(checkpoint_dir, exist_ok=True)
    os.makedirs(log_dir, exist_ok=True)

    return [
        # Early Stopping
        EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True,
            min_delta=0.0001
        ),
        # Reduce LR on Plateau
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5,
            min_lr=1e-6,
            verbose=1
        ),
        # Model Checkpoint
        ModelCheckpoint(
            filepath=os.path.join(checkpoint_dir, 'best_model_{epoch:02d}_{val_loss:.4f}.h5'),
            monitor='val_loss',
            save_best_only=True,
            save_weights_only=True,
            verbose=1
        ),
        # TensorBoard
        tf.keras.callbacks.TensorBoard(
            log_dir=log_dir,
            histogram_freq=1,
            write_graph=True,
            update_freq='epoch'
        ),
        # Custom callback
        CustomCallback()
    ]


def train_radiation_model(X_train, y_train, X_val, y_val, solar_params_train, solar_params_val,
                          scalers=None, **kwargs):
    """
    Addestra il modello per la radiazione solare
    
    Parameters:
    -----------
    X_train : array-like
        Dati di input per il training
    y_train : array-like
        Target per il training
    X_val : array-like
        Dati di input per la validazione
    y_val : array-like
        Target per la validazione
    solar_params_train : array-like
        Parametri solari per il training
    solar_params_val : array-like
        Parametri solari per la validazione
    scalers : dict, optional
        Dizionario degli scaler (es. {'X': x_scaler, 'y': y_scaler, 'solar_params': solar_params_scaler})
    **kwargs : dict
        Parametri aggiuntivi per il training (epochs, batch_size, etc.)
    
    Returns:
    --------
    tuple
        (model, history)
    """
    print("\nAddestramento modello Radiation...")

    # Crea e compila il modello
    model = create_radiation_model(input_shape=X_train.shape[1:])
    model.compile(
        optimizer='adam',
        loss='mse',
        metrics=['mae', 'mse']
    )

    # Mostra il summary del modello
    model.summary()

    # Addestra il modello
    history = model.fit(
        [X_train, solar_params_train],
        y_train,
        validation_data=([X_val, solar_params_val], y_val),
        **kwargs
    )

    # Salva il modello con tutti gli artefatti
    save_single_model_and_scalers(
        model=model,
        model_name='solarradiation',
        scalers=scalers,
        base_path='./models'
    )

    print("\nAddestramento completato e modello salvato!")
    return model, history


def train_energy_model(X_train, y_train, X_val, y_val, scalers=None, **kwargs):
    """
    Addestra il modello per l'energia solare
    
    Parameters:
    -----------
    X_train : array-like
        Dati di input per il training
    y_train : array-like
        Target per il training
    X_val : array-like
        Dati di input per la validazione
    y_val : array-like
        Target per la validazione
    scalers : dict, optional
        Dizionario degli scaler (es. {'X': x_scaler, 'y': y_scaler})
    **kwargs : dict
        Parametri aggiuntivi per il training (epochs, batch_size, etc.)
    
    Returns:
    --------
    tuple
        (model, history)
    """
    print("\nAddestramento modello Energy...")

    # Crea e compila il modello
    model = create_energy_model(input_shape=X_train.shape[1:])
    model.compile(
        optimizer='adam',
        loss='mse',
        metrics=['mae', 'mse']
    )

    # Mostra il summary del modello
    model.summary()

    # Addestra il modello
    history = model.fit(
        X_train,
        y_train,
        validation_data=(X_val, y_val),
        **kwargs
    )

    # Salva il modello con tutti gli artefatti
    save_single_model_and_scalers(
        model=model,
        model_name='solarenergy',
        scalers=scalers,
        base_path='./models'
    )

    print("\nAddestramento completato e modello salvato!")
    return model, history


def train_uv_model(X_train, y_train, X_val, y_val, scalers=None, **kwargs):
    """
    Addestra il modello per l'indice UV
    
    Parameters:
    -----------
    X_train : array-like
        Dati di input per il training
    y_train : array-like
        Target per il training
    X_val : array-like
        Dati di input per la validazione
    y_val : array-like
        Target per la validazione
    scalers : dict, optional
        Dizionario degli scaler (es. {'X': x_scaler, 'y': y_scaler})
    **kwargs : dict
        Parametri aggiuntivi per il training (epochs, batch_size, etc.)
    
    Returns:
    --------
    tuple
        (model, history)
    """
    print("\nAddestramento modello UV...")

    # Crea e compila il modello
    model = create_uv_model(input_shape=X_train.shape[1:])
    model.compile(
        optimizer='adam',
        loss='mse',
        metrics=['mae', 'mse']
    )

    # Mostra il summary del modello
    model.summary()

    # Addestra il modello
    history = model.fit(
        X_train,
        y_train,
        validation_data=(X_val, y_val),
        **kwargs
    )

    # Salva il modello con tutti gli artefatti
    save_single_model_and_scalers(
        model=model,
        model_name='uvindex',
        scalers=scalers,
        base_path='./models'
    )

    print("\nAddestramento completato e modello salvato!")
    return model, history


def save_single_model_and_scalers(model, model_name, scalers=None, base_path='./kaggle/working/models'):
    """
    Salva un singolo modello con tutti i suoi artefatti associati e multipli scaler.
    
    Parameters:
    -----------
    model : keras.Model
        Il modello da salvare
    model_name : str
        Nome del modello (es. 'solarradiation', 'solarenergy', 'uvindex')
    scalers : dict, optional
        Dizionario degli scaler associati al modello (es. {'X': x_scaler, 'y': y_scaler})
    base_path : str
        Percorso base dove salvare il modello
    """
    if isinstance(base_path, list):
        base_path = './kaggle/working/models'

    # Crea la cartella base se non esiste
    os.makedirs(base_path, exist_ok=True)

    # Crea la sottocartella per il modello specifico
    model_path = os.path.join(base_path, model_name)
    os.makedirs(model_path, exist_ok=True)

    try:
        print(f"\nSalvataggio modello {model_name}...")

        # 1. Salva il modello completo
        model_file = os.path.join(model_path, 'model.keras')
        model.save(model_file, save_format='keras')
        print(f"- Salvato modello completo: {model_file}")

        # 2. Salva i pesi separatamente
        weights_path = os.path.join(model_path, 'weights')
        os.makedirs(weights_path, exist_ok=True)
        weight_file = os.path.join(weights_path, 'weights')
        model.save_weights(weight_file)
        print(f"- Salvati pesi: {weight_file}")

        # 3. Salva il plot del modello
        plot_path = os.path.join(model_path, f'{model_name}_architecture.png')
        tf.keras.utils.plot_model(
            model,
            to_file=plot_path,
            show_shapes=True,
            show_layer_names=True,
            rankdir='TB',
            expand_nested=True,
            dpi=150
        )
        print(f"- Salvato plot architettura: {plot_path}")

        # 4. Salva il summary del modello
        summary_path = os.path.join(model_path, f'{model_name}_summary.txt')
        with open(summary_path, 'w') as f:
            model.summary(print_fn=lambda x: f.write(x + '\n'))
        print(f"- Salvato summary modello: {summary_path}")

        # 5. Salva gli scaler se forniti
        if scalers is not None:
            scaler_path = os.path.join(model_path, 'scalers')
            os.makedirs(scaler_path, exist_ok=True)

            for scaler_name, scaler in scalers.items():
                scaler_file = os.path.join(scaler_path, f'{scaler_name}_scaler.joblib')
                joblib.dump(scaler, scaler_file)
                print(f"- Salvato scaler {scaler_name}: {scaler_file}")

        # 6. Salva la configurazione del modello
        model_config = {
            'has_solar_params': True if model_name == 'solarradiation' else False,
            'scalers': list(scalers.keys()) if scalers else []
        }
        config_path = os.path.join(model_path, 'model_config.joblib')
        joblib.dump(model_config, config_path)
        print(f"- Salvata configurazione: {config_path}")

        # 7. Crea un README specifico per il modello
        readme_path = os.path.join(model_path, 'README.txt')
        with open(readme_path, 'w') as f:
            f.write(f"{model_name.upper()} Model Artifacts\n")
            f.write("=" * (len(model_name) + 15) + "\n\n")
            f.write("Directory structure:\n")
            f.write("- model.keras: Complete model\n")
            f.write("- weights/: Model weights\n")
            f.write(f"- {model_name}_architecture.png: Visual representation of model architecture\n")
            f.write(f"- {model_name}_summary.txt: Detailed model summary\n")
            f.write("- model_config.joblib: Model configuration\n")
            if scalers:
                f.write("- scalers/: Directory containing model scalers\n")
                for scaler_name in scalers.keys():
                    f.write(f"  - {scaler_name}_scaler.joblib: {scaler_name} scaler\n")

        print(f"\nTutti gli artefatti per {model_name} salvati in: {model_path}")
        print(f"Consulta {readme_path} per i dettagli sulla struttura")

    except Exception as e:
        print(f"Errore nel salvataggio degli artefatti per {model_name}: {str(e)}")
        raise

    return model_path


def load_single_model_and_scalers(model_name, base_path='./kaggle/working/models'):
    """
    Carica un singolo modello con tutti i suoi artefatti e scaler associati.
    
    Parameters:
    -----------
    model_name : str
        Nome del modello da caricare (es. 'solarradiation', 'solarenergy', 'uvindex')
    base_path : str
        Percorso base dove sono salvati i modelli
        
    Returns:
    --------
    tuple
        (model, scalers, model_config)
    """
    model_path = os.path.join(base_path, model_name)

    if not os.path.exists(model_path):
        print(f"Directory del modello non trovata: {model_path}")
        return None, None, None

    try:
        print(f"\nCaricamento modello {model_name}...")

        # 1. Carica la configurazione del modello
        config_path = os.path.join(model_path, 'model_config.joblib')
        try:
            model_config = joblib.load(config_path)
            print("- Configurazione modello caricata")
        except:
            print("! Configurazione modello non trovata, usando configurazione di default")
            model_config = {
                'has_solar_params': True if model_name == 'solarradiation' else False,
                'scalers': ['X', 'y']
            }

        # 2. Carica il modello
        try:
            # Prima prova a caricare il modello completo
            model_file = os.path.join(model_path, 'model.keras')
            model = tf.keras.models.load_model(model_file)
            print(f"- Modello caricato da: {model_file}")

            # Verifica i pesi
            weights_path = os.path.join(model_path, 'weights', 'weights')
            if os.path.exists(weights_path + '.index'):
                model.load_weights(weights_path)
                print("- Pesi verificati con successo")

        except Exception as e:
            print(f"! Errore nel caricamento del modello: {str(e)}")
            print("Tentativo di ricostruzione del modello...")

            try:
                # Ricostruzione del modello
                if model_name == 'solarradiation':
                    model = create_radiation_model(input_shape=(24, 8))
                elif model_name == 'solarenergy':
                    model = create_energy_model(input_shape=(24, 8))
                elif model_name == 'uvindex':
                    model = create_uv_model(input_shape=(24, 8))
                else:
                    raise ValueError(f"Tipo di modello non riconosciuto: {model_name}")

                # Carica i pesi
                model.load_weights(weights_path)
                print("- Modello ricostruito dai pesi con successo")
            except Exception as e:
                print(f"! Errore nella ricostruzione del modello: {str(e)}")
                return None, None, None

        # 3. Carica gli scaler
        scalers = {}
        scaler_path = os.path.join(model_path, 'scalers')
        if os.path.exists(scaler_path):
            print("\nCaricamento scaler:")
            for scaler_file in os.listdir(scaler_path):
                if scaler_file.endswith('_scaler.joblib'):
                    scaler_name = scaler_file.replace('_scaler.joblib', '')
                    scaler_file_path = os.path.join(scaler_path, scaler_file)
                    try:
                        scalers[scaler_name] = joblib.load(scaler_file_path)
                        print(f"- Caricato scaler {scaler_name}")
                    except Exception as e:
                        print(f"! Errore nel caricamento dello scaler {scaler_name}: {str(e)}")
        else:
            print("! Directory degli scaler non trovata")

        # 4. Verifica integrità del modello
        try:
            # Verifica che il modello possa fare predizioni
            if model_name == 'solarradiation':
                dummy_input = [np.zeros((1, 24, 8)), np.zeros((1, 3))]
            else:
                dummy_input = np.zeros((1, 24, 8))

            model.predict(dummy_input, verbose=0)
            print("\n✓ Verifica integrità modello completata con successo")
        except Exception as e:
            print(f"\n! Attenzione: il modello potrebbe non funzionare correttamente: {str(e)}")

        # 5. Carica e verifica il summary del modello
        summary_path = os.path.join(model_path, f'{model_name}_summary.txt')
        if os.path.exists(summary_path):
            print("\nSummary del modello disponibile in:", summary_path)

        # 6. Verifica il plot dell'architettura
        plot_path = os.path.join(model_path, f'{model_name}_architecture.png')
        if os.path.exists(plot_path):
            print("Plot dell'architettura disponibile in:", plot_path)

        print(f"\nCaricamento di {model_name} completato con successo!")
        return model, scalers, model_config

    except Exception as e:
        print(f"\nErrore critico nel caricamento del modello {model_name}: {str(e)}")
        return None, None, None

In [ ]:
def process_predictions(predictions, scaler, dates, data, target):
    """
    Processa e salva le predizioni nel DataFrame.
    
    Parameters:
    -----------
    predictions : array-like
        Predizioni scalate
    scaler : object
        Scaler per denormalizzare le predizioni
    dates : array-like
        Date per le predizioni
    data : pd.DataFrame
        DataFrame da aggiornare
    target : str
        Nome della variabile target
    
    Returns:
    --------
    array-like
        Predizioni denormalizzate
    """
    # Verifica e gestione NaN nelle predizioni
    if np.isnan(predictions).any():
        print("ATTENZIONE: Trovati NaN nelle predizioni")
        predictions = np.nan_to_num(predictions, 0)

    # Denormalizza e applica vincolo di non negatività
    y_pred = scaler.inverse_transform(predictions)
    y_pred = np.maximum(y_pred, 0)

    # Aggiorna il DataFrame
    if len(dates) > len(y_pred):
        dates = dates[:len(y_pred)]
    data.loc[dates, target] = y_pred

    # Stampa statistiche
    print(f"\nStatistiche predizioni per {target}:")
    print(f"Media: {np.mean(y_pred):.2f}")
    print(f"Min: {np.min(y_pred):.2f}")
    print(f"Max: {np.max(y_pred):.2f}")

    return y_pred


def predict_radiation(data, features, model, scalers, timesteps=24):
    """
    Effettua predizioni per la radiazione solare.
    """
    print("\nPredizione Radiazione Solare")
    print("=" * 50)

    try:
        # Prepara features base
        X_current = scalers['X'].transform(data[features].values)
        X_seq = create_sequences(timesteps, X_current)

        # Prepara parametri solari
        solar_columns = ['solar_angle', 'clear_sky_index', 'solar_elevation']
        solar_params = data[solar_columns].values
        solar_params_seq = solar_params[timesteps:len(X_seq) + timesteps]

        # Effettua predizioni
        predictions = model.predict(
            [X_seq, solar_params_seq],
            batch_size=32,
            verbose=1
        )

        # Processa e salva predizioni
        dates = data.index[timesteps:]
        return process_predictions(predictions, scalers['solarradiation'], dates, data, 'solarradiation')

    except Exception as e:
        print(f"Errore nella predizione della radiazione: {str(e)}")
        return np.zeros(len(data) - timesteps)


def predict_energy(data, features, model, scalers, radiation_pred=None, timesteps=24):
    """
    Effettua predizioni per l'energia solare.
    """
    print("\nPredizione Energia Solare")
    print("=" * 50)

    try:
        # Prepara features base
        X_current = scalers['X'].transform(data[features].values)

        # Aggiungi predizioni della radiazione se disponibili
        if radiation_pred is not None:
            radiation_scaled = scalers['solarradiation'].transform(radiation_pred.reshape(-1, 1))
            X_current = np.column_stack([X_current, radiation_scaled])

        X_seq = create_sequences(timesteps, X_current)

        # Effettua predizioni
        predictions = model.predict(X_seq, batch_size=32, verbose=1)

        # Processa e salva predizioni
        dates = data.index[timesteps:]
        return process_predictions(predictions, scalers['solarenergy'], dates, data, 'solarenergy')

    except Exception as e:
        print(f"Errore nella predizione dell'energia: {str(e)}")
        return np.zeros(len(data) - timesteps)


def predict_uv(data, features, model, scalers, radiation_pred=None, energy_pred=None, timesteps=24):
    """
    Effettua predizioni per l'indice UV.
    """
    print("\nPredizione Indice UV")
    print("=" * 50)

    try:
        # Prepara features base
        X_current = scalers['X'].transform(data[features].values)

        # Aggiungi predizioni precedenti se disponibili
        if radiation_pred is not None:
            radiation_scaled = scalers['solarradiation'].transform(radiation_pred.reshape(-1, 1))
            X_current = np.column_stack([X_current, radiation_scaled])

        if energy_pred is not None:
            energy_scaled = scalers['solarenergy'].transform(energy_pred.reshape(-1, 1))
            X_current = np.column_stack([X_current, energy_scaled])

        X_seq = create_sequences(timesteps, X_current)

        # Effettua predizioni
        predictions = model.predict(X_seq, batch_size=32, verbose=1)

        # Processa e salva predizioni
        dates = data.index[timesteps:]
        return process_predictions(predictions, scalers['uvindex'], dates, data, 'uvindex')

    except Exception as e:
        print(f"Errore nella predizione dell'UV: {str(e)}")
        return np.zeros(len(data) - timesteps)

In [ ]:
def add_olive_water_consumption_correlation(dataset):
    # Dati simulati per il fabbisogno d'acqua e la correlazione con la temperatura
    fabbisogno_acqua = {
        "Nocellara dell'Etna": {"Primavera": 1200, "Estate": 2000, "Autunno": 1000, "Inverno": 500, "Temperatura Ottimale": 18, "Resistenza": "Media"},
        "Leccino": {"Primavera": 1000, "Estate": 1800, "Autunno": 800, "Inverno": 400, "Temperatura Ottimale": 20, "Resistenza": "Alta"},
        "Frantoio": {"Primavera": 1100, "Estate": 1900, "Autunno": 900, "Inverno": 450, "Temperatura Ottimale": 19, "Resistenza": "Alta"},
        "Coratina": {"Primavera": 1300, "Estate": 2200, "Autunno": 1100, "Inverno": 550, "Temperatura Ottimale": 17, "Resistenza": "Media"},
        "Moraiolo": {"Primavera": 1150, "Estate": 2100, "Autunno": 900, "Inverno": 480, "Temperatura Ottimale": 18, "Resistenza": "Media"},
        "Pendolino": {"Primavera": 1050, "Estate": 1850, "Autunno": 850, "Inverno": 430, "Temperatura Ottimale": 20, "Resistenza": "Alta"},
        "Taggiasca": {"Primavera": 1000, "Estate": 1750, "Autunno": 800, "Inverno": 400, "Temperatura Ottimale": 19, "Resistenza": "Alta"},
        "Canino": {"Primavera": 1100, "Estate": 1900, "Autunno": 900, "Inverno": 450, "Temperatura Ottimale": 18, "Resistenza": "Media"},
        "Itrana": {"Primavera": 1200, "Estate": 2000, "Autunno": 1000, "Inverno": 500, "Temperatura Ottimale": 17, "Resistenza": "Media"},
        "Ogliarola": {"Primavera": 1150, "Estate": 1950, "Autunno": 900, "Inverno": 480, "Temperatura Ottimale": 18, "Resistenza": "Media"},
        "Biancolilla": {"Primavera": 1050, "Estate": 1800, "Autunno": 850, "Inverno": 430, "Temperatura Ottimale": 19, "Resistenza": "Alta"}
    }

    # Calcola il fabbisogno idrico annuale per ogni varietà
    for varieta in fabbisogno_acqua:
        fabbisogno_acqua[varieta]["Annuale"] = sum([fabbisogno_acqua[varieta][stagione] for stagione in ["Primavera", "Estate", "Autunno", "Inverno"]])

    # Aggiungiamo le nuove colonne al dataset
    dataset["Fabbisogno Acqua Primavera (m³/ettaro)"] = dataset["Varietà di Olive"].apply(lambda x: fabbisogno_acqua[x]["Primavera"])
    dataset["Fabbisogno Acqua Estate (m³/ettaro)"] = dataset["Varietà di Olive"].apply(lambda x: fabbisogno_acqua[x]["Estate"])
    dataset["Fabbisogno Acqua Autunno (m³/ettaro)"] = dataset["Varietà di Olive"].apply(lambda x: fabbisogno_acqua[x]["Autunno"])
    dataset["Fabbisogno Acqua Inverno (m³/ettaro)"] = dataset["Varietà di Olive"].apply(lambda x: fabbisogno_acqua[x]["Inverno"])
    dataset["Fabbisogno Idrico Annuale (m³/ettaro)"] = dataset["Varietà di Olive"].apply(lambda x: fabbisogno_acqua[x]["Annuale"])
    dataset["Temperatura Ottimale"] = dataset["Varietà di Olive"].apply(lambda x: fabbisogno_acqua[x]["Temperatura Ottimale"])
    dataset["Resistenza alla Siccità"] = dataset["Varietà di Olive"].apply(lambda x: fabbisogno_acqua[x]["Resistenza"])

    return dataset

In [ ]:
def preprocess_weather_data(weather_df):
    # Calcola statistiche mensili per ogni anno
    monthly_weather = weather_df.groupby(['year', 'month']).agg({
        'temp': ['mean', 'min', 'max'],
        'humidity': 'mean',
        'precip': 'sum',
        'windspeed': 'mean',
        'cloudcover': 'mean',
        'solarradiation': 'sum',
        'solarenergy': 'sum',
        'uvindex': 'max'
    }).reset_index()

    monthly_weather.columns = ['year', 'month'] + [f'{col[0]}_{col[1]}' for col in monthly_weather.columns[2:]]
    return monthly_weather


def get_growth_phase(month):
    if month in [12, 1, 2]:
        return 'dormancy'
    elif month in [3, 4, 5]:
        return 'flowering'
    elif month in [6, 7, 8]:
        return 'fruit_set'
    else:
        return 'ripening'


def calculate_weather_effect(row, optimal_temp):
    # Effetti base
    temp_effect = -0.1 * (row['temp_mean'] - optimal_temp) ** 2
    rain_effect = -0.05 * (row['precip_sum'] - 600) ** 2 / 10000
    sun_effect = 0.1 * row['solarenergy_sum'] / 1000

    # Fattori di scala basati sulla fase di crescita
    if row['growth_phase'] == 'dormancy':
        temp_scale = 0.5
        rain_scale = 0.2
        sun_scale = 0.1
    elif row['growth_phase'] == 'flowering':
        temp_scale = 2.0
        rain_scale = 1.5
        sun_scale = 1.0
    elif row['growth_phase'] == 'fruit_set':
        temp_scale = 1.5
        rain_scale = 1.0
        sun_scale = 0.8
    else:  # ripening
        temp_scale = 1.0
        rain_scale = 0.5
        sun_scale = 1.2

    # Calcolo dell'effetto combinato
    combined_effect = (
            temp_scale * temp_effect +
            rain_scale * rain_effect +
            sun_scale * sun_effect
    )

    # Aggiustamenti specifici per fase
    if row['growth_phase'] == 'flowering':
        combined_effect -= 0.5 * max(0, row['precip_sum'] - 50)  # Penalità per pioggia eccessiva durante la fioritura
    elif row['growth_phase'] == 'fruit_set':
        combined_effect += 0.3 * max(0, row['temp_mean'] - (optimal_temp + 5))  # Bonus per temperature più alte durante la formazione dei frutti

    return combined_effect


def calculate_water_need(weather_data, base_need, optimal_temp):
    # Calcola il fabbisogno idrico basato su temperatura e precipitazioni
    temp_factor = 1 + 0.05 * (weather_data['temp_mean'] - optimal_temp)  # Aumenta del 5% per ogni grado sopra l'ottimale
    rain_factor = 1 - 0.001 * weather_data['precip_sum']  # Diminuisce leggermente con l'aumentare delle precipitazioni
    return base_need * temp_factor * rain_factor


def clean_column_name(name):
    # Rimuove caratteri speciali e spazi, converte in snake_case e abbrevia
    name = re.sub(r'[^a-zA-Z0-9\s]', '', name)  # Rimuove caratteri speciali
    name = name.lower().replace(' ', '_')  # Converte in snake_case

    # Abbreviazioni comuni
    abbreviations = {
        'production': 'prod',
        'percentage': 'pct',
        'hectare': 'ha',
        'tonnes': 't',
        'litres': 'l',
        'minimum': 'min',
        'maximum': 'max',
        'average': 'avg'
    }

    for full, abbr in abbreviations.items():
        name = name.replace(full, abbr)

    return name


def create_technique_mapping(olive_varieties, mapping_path='./kaggle/working/models/technique_mapping.joblib'):
    # Estrai tutte le tecniche uniche dal dataset e convertile in lowercase
    all_techniques = olive_varieties['Tecnica di Coltivazione'].str.lower().unique()

    # Crea il mapping partendo da 1
    technique_mapping = {tech: i + 1 for i, tech in enumerate(sorted(all_techniques))}

    # Salva il mapping
    os.makedirs(os.path.dirname(mapping_path), exist_ok=True)
    joblib.dump(technique_mapping, mapping_path)

    return technique_mapping


def encode_techniques(df, mapping_path='./kaggle/working/models/technique_mapping.joblib'):
    if not os.path.exists(mapping_path):
        raise FileNotFoundError(f"Mapping not found at {mapping_path}. Run create_technique_mapping first.")

    technique_mapping = joblib.load(mapping_path)

    # Trova tutte le colonne delle tecniche
    tech_columns = [col for col in df.columns if col.endswith('_tech')]

    # Applica il mapping a tutte le colonne delle tecniche
    for col in tech_columns:
        df[col] = df[col].str.lower().map(technique_mapping).fillna(0).astype(int)

    return df


def decode_techniques(df, mapping_path='./kaggle/working/models/technique_mapping.joblib'):
    if not os.path.exists(mapping_path):
        raise FileNotFoundError(f"Mapping not found at {mapping_path}")

    technique_mapping = joblib.load(mapping_path)
    reverse_mapping = {v: k for k, v in technique_mapping.items()}
    reverse_mapping[0] = ''  # Aggiungi un mapping per 0 a stringa vuota

    # Trova tutte le colonne delle tecniche
    tech_columns = [col for col in df.columns if col.endswith('_tech')]

    # Applica il reverse mapping a tutte le colonne delle tecniche
    for col in tech_columns:
        df[col] = df[col].map(reverse_mapping)

    return df


def decode_single_technique(technique_value, mapping_path='./kaggle/working/models/technique_mapping.joblib'):
    if not os.path.exists(mapping_path):
        raise FileNotFoundError(f"Mapping not found at {mapping_path}")

    technique_mapping = joblib.load(mapping_path)
    reverse_mapping = {v: k for k, v in technique_mapping.items()}
    reverse_mapping[0] = ''

    return reverse_mapping.get(technique_value, '')

In [ ]:
def get_optimal_workers():
    """
    Calcola il numero ottimale di workers basandosi sulle risorse del sistema.
    
    Returns:
        int: Numero ottimale di workers
    """
    # Ottiene il numero di CPU logiche (inclusi i thread virtuali)
    cpu_count = multiprocessing.cpu_count()

    # Ottiene la memoria totale e disponibile in GB
    memory = psutil.virtual_memory()
    total_memory_gb = memory.total / (1024 ** 3)
    available_memory_gb = memory.available / (1024 ** 3)

    # Stima della memoria necessaria per worker (esempio: 2GB per worker)
    memory_per_worker_gb = 2

    # Calcola il numero massimo di workers basato sulla memoria disponibile
    max_workers_by_memory = int(available_memory_gb / memory_per_worker_gb)

    # Usa il minimo tra:
    # - numero di CPU disponibili - 1 (lascia una CPU libera per il sistema)
    # - numero massimo di workers basato sulla memoria
    # - un limite massimo arbitrario (es. 16) per evitare troppo overhead
    optimal_workers = min(
        cpu_count - 1,
        max_workers_by_memory,
        32  # limite massimo arbitrario
    )

    # Assicura almeno 1 worker
    return max(1, optimal_workers)


def simulate_zone(base_weather, olive_varieties, year, zone, all_varieties, variety_techniques):
    """
    Simula la produzione di olive per una singola zona.
    
    Args:
        base_weather: DataFrame con dati meteo di base per l'anno selezionato
        olive_varieties: DataFrame con le informazioni sulle varietà di olive
        zone: ID della zona
        all_varieties: Array con tutte le varietà disponibili
        variety_techniques: Dict con le tecniche disponibili per ogni varietà
    
    Returns:
        Dict con i risultati della simulazione per la zona
    """
    # Crea una copia dei dati meteo per questa zona specifica
    zone_weather = base_weather.copy()

    # Genera variazioni meteorologiche specifiche per questa zona
    zone_weather['temp_mean'] *= np.random.uniform(0.95, 1.05, len(zone_weather))
    zone_weather['precip_sum'] *= np.random.uniform(0.9, 1.1, len(zone_weather))
    zone_weather['solarenergy_sum'] *= np.random.uniform(0.95, 1.05, len(zone_weather))

    # Genera caratteristiche specifiche della zona
    num_varieties = np.random.randint(1, 4)  # 1-3 varietà per zona
    selected_varieties = np.random.choice(all_varieties, size=num_varieties, replace=False)
    hectares = np.random.uniform(1, 10)  # Dimensione del terreno
    percentages = np.random.dirichlet(np.ones(num_varieties))  # Distribuzione delle varietà

    # Inizializzazione contatori annuali
    annual_production = 0
    annual_min_oil = 0
    annual_max_oil = 0
    annual_avg_oil = 0
    annual_water_need = 0

    # Inizializzazione dizionario dati varietà
    variety_data = {clean_column_name(variety): {
        'tech': '',
        'pct': 0,
        'prod_t_ha': 0,
        'oil_prod_t_ha': 0,
        'oil_prod_l_ha': 0,
        'min_yield_pct': 0,
        'max_yield_pct': 0,
        'min_oil_prod_l_ha': 0,
        'max_oil_prod_l_ha': 0,
        'avg_oil_prod_l_ha': 0,
        'l_per_t': 0,
        'min_l_per_t': 0,
        'max_l_per_t': 0,
        'avg_l_per_t': 0,
        'olive_prod': 0,
        'min_oil_prod': 0,
        'max_oil_prod': 0,
        'avg_oil_prod': 0,
        'water_need': 0
    } for variety in all_varieties}

    # Simula produzione per ogni varietà selezionata
    for i, variety in enumerate(selected_varieties):
        # Seleziona tecnica di coltivazione casuale per questa varietà
        technique = np.random.choice(variety_techniques[variety])
        percentage = percentages[i]

        # Ottieni informazioni specifiche della varietà
        variety_info = olive_varieties[
            (olive_varieties['Varietà di Olive'] == variety) &
            (olive_varieties['Tecnica di Coltivazione'] == technique)
            ].iloc[0]

        # Calcola produzione base con variabilità
        base_production = variety_info['Produzione (tonnellate/ettaro)'] * 1000 * percentage * hectares / 12
        base_production *= np.random.uniform(0.9, 1.1)

        # Calcola effetti meteo sulla produzione
        weather_effect = zone_weather.apply(
            lambda row: calculate_weather_effect(row, variety_info['Temperatura Ottimale']),
            axis=1
        )
        monthly_production = base_production * (1 + weather_effect / 10000)
        monthly_production *= np.random.uniform(0.95, 1.05, len(zone_weather))

        # Calcola produzione annuale per questa varietà
        annual_variety_production = monthly_production.sum()

        # Calcola rese di olio con variabilità
        min_yield_factor = np.random.uniform(0.95, 1.05)
        max_yield_factor = np.random.uniform(0.95, 1.05)
        avg_yield_factor = (min_yield_factor + max_yield_factor) / 2

        min_oil_production = annual_variety_production * variety_info['Min Litri per Tonnellata'] / 1000 * min_yield_factor
        max_oil_production = annual_variety_production * variety_info['Max Litri per Tonnellata'] / 1000 * max_yield_factor
        avg_oil_production = annual_variety_production * variety_info['Media Litri per Tonnellata'] / 1000 * avg_yield_factor

        # Calcola fabbisogno idrico
        base_water_need = (
                                  variety_info['Fabbisogno Acqua Primavera (m³/ettaro)'] +
                                  variety_info['Fabbisogno Acqua Estate (m³/ettaro)'] +
                                  variety_info['Fabbisogno Acqua Autunno (m³/ettaro)'] +
                                  variety_info['Fabbisogno Acqua Inverno (m³/ettaro)']
                          ) / 4

        monthly_water_need = zone_weather.apply(
            lambda row: calculate_water_need(row, base_water_need, variety_info['Temperatura Ottimale']),
            axis=1
        )
        monthly_water_need *= np.random.uniform(0.95, 1.05, len(monthly_water_need))
        annual_variety_water_need = monthly_water_need.sum() * percentage * hectares

        # Aggiorna totali annuali
        annual_production += annual_variety_production
        annual_min_oil += min_oil_production
        annual_max_oil += max_oil_production
        annual_avg_oil += avg_oil_production
        annual_water_need += annual_variety_water_need

        # Aggiorna dati varietà
        clean_variety = clean_column_name(variety)
        variety_data[clean_variety].update({
            'tech': clean_column_name(technique),
            'pct': percentage,
            'prod_t_ha': variety_info['Produzione (tonnellate/ettaro)'] * np.random.uniform(0.95, 1.05),
            'oil_prod_t_ha': variety_info['Produzione Olio (tonnellate/ettaro)'] * np.random.uniform(0.95, 1.05),
            'oil_prod_l_ha': variety_info['Produzione Olio (litri/ettaro)'] * np.random.uniform(0.95, 1.05),
            'min_yield_pct': variety_info['Min % Resa'] * min_yield_factor,
            'max_yield_pct': variety_info['Max % Resa'] * max_yield_factor,
            'min_oil_prod_l_ha': variety_info['Min Produzione Olio (litri/ettaro)'] * min_yield_factor,
            'max_oil_prod_l_ha': variety_info['Max Produzione Olio (litri/ettaro)'] * max_yield_factor,
            'avg_oil_prod_l_ha': variety_info['Media Produzione Olio (litri/ettaro)'] * avg_yield_factor,
            'l_per_t': variety_info['Litri per Tonnellata'] * np.random.uniform(0.98, 1.02),
            'min_l_per_t': variety_info['Min Litri per Tonnellata'] * min_yield_factor,
            'max_l_per_t': variety_info['Max Litri per Tonnellata'] * max_yield_factor,
            'avg_l_per_t': variety_info['Media Litri per Tonnellata'] * avg_yield_factor,
            'olive_prod': annual_variety_production,
            'min_oil_prod': min_oil_production,
            'max_oil_prod': max_oil_production,
            'avg_oil_prod': avg_oil_production,
            'water_need': annual_variety_water_need
        })

    # Appiattisci i dati delle varietà
    flattened_variety_data = {
        f'{variety}_{key}': value
        for variety, data in variety_data.items()
        for key, value in data.items()
    }

    # Restituisci il risultato della zona
    return {
        'year': year,
        'zone_id': zone + 1,
        'temp_mean': zone_weather['temp_mean'].mean(),
        'precip_sum': zone_weather['precip_sum'].sum(),
        'solar_energy_sum': zone_weather['solarenergy_sum'].sum(),
        'ha': hectares,
        'zone': f"zone_{zone + 1}",
        'olive_prod': annual_production,
        'min_oil_prod': annual_min_oil,
        'max_oil_prod': annual_max_oil,
        'avg_oil_prod': annual_avg_oil,
        'total_water_need': annual_water_need,
        **flattened_variety_data
    }


def simulate_olive_production_parallel(weather_data, olive_varieties, num_simulations=5,
                                       random_seed=None, max_workers=None, batch_size=500,
                                       output_path="./kaggle/working/data/simulated_data.parquet"):
    """
    Versione ottimizzata della simulazione che salva i risultati in un unico file parquet partizionato
    
    Parameters:
    -----------
    weather_data : DataFrame
        Dati meteorologici di input
    olive_varieties : DataFrame
        Dati sulle varietà di olive
    num_simulations : int
        Numero totale di simulazioni da eseguire
    random_seed : int, optional
        Seed per la riproducibilità
    max_workers : int, optional
        Numero massimo di workers per la parallelizzazione
    batch_size : int
        Dimensione di ogni batch di simulazioni
    output_path : str
        Percorso del file parquet di output (includerà le partizioni)
    """
    import os
    from math import ceil

    if random_seed is not None:
        np.random.seed(random_seed)

    # Preparazione dati
    create_technique_mapping(olive_varieties)
    monthly_weather = preprocess_weather_data(weather_data)
    all_varieties = olive_varieties['Varietà di Olive'].unique()
    variety_techniques = {
        variety: olive_varieties[olive_varieties['Varietà di Olive'] == variety]['Tecnica di Coltivazione'].unique()
        for variety in all_varieties
    }

    # Calcolo workers ottimali se non specificati
    if max_workers is None:
        max_workers = get_optimal_workers() or 1
        print(f"Utilizzando {max_workers} workers basati sulle risorse del sistema")

    # Calcolo del numero di batch necessari
    num_batches = ceil(num_simulations / batch_size)
    print(f"Elaborazione di {num_simulations} simulazioni in {num_batches} batch")

    # Crea directory parent se non esiste
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    for batch_num in range(num_batches):
        start_sim = batch_num * batch_size
        end_sim = min((batch_num + 1) * batch_size, num_simulations)
        current_batch_size = end_sim - start_sim

        batch_results = []

        # Parallelizzazione usando ProcessPoolExecutor
        with ProcessPoolExecutor(max_workers=max_workers) as executor:
            with tqdm(total=current_batch_size * current_batch_size,
                      desc=f"Batch {batch_num + 1}/{num_batches}") as pbar:

                future_to_sim_id = {}

                # Sottometti i lavori per il batch corrente
                for sim in range(start_sim, end_sim):
                    selected_year = np.random.choice(monthly_weather['year'].unique())
                    base_weather = monthly_weather[monthly_weather['year'] == selected_year].copy()
                    base_weather.loc[:, 'growth_phase'] = base_weather['month'].apply(get_growth_phase)

                    for zone in range(current_batch_size):
                        future = executor.submit(
                            simulate_zone,
                            base_weather=base_weather,
                            olive_varieties=olive_varieties,
                            year=selected_year,
                            zone=zone,
                            all_varieties=all_varieties,
                            variety_techniques=variety_techniques
                        )
                        future_to_sim_id[future] = sim + 1

                # Raccogli i risultati del batch
                for future in as_completed(future_to_sim_id.keys()):
                    sim_id = future_to_sim_id[future]
                    try:
                        result = future.result()
                        result['simulation_id'] = sim_id
                        result['batch_id'] = batch_num  # Aggiungiamo batch_id per il partizionamento
                        batch_results.append(result)
                        pbar.update(1)
                    except Exception as e:
                        print(f"Errore nella simulazione {sim_id}: {str(e)}")
                        continue

        # Converti i risultati del batch in DataFrame
        batch_df = pd.DataFrame(batch_results)

        # Salva il batch come partizione del file parquet
        batch_df.to_parquet(
            output_path,
            partition_cols=['batch_id'],  # Partiziona per batch_id
            append=batch_num > 0  # Appendi se non è il primo batch
        )

        # Libera memoria
        del batch_results
        del batch_df

    print(f"Simulazione completata. I dati sono stati salvati in: {output_path}")


# Funzione per visualizzare il mapping delle tecniche
def print_technique_mapping(mapping_path='./kaggle/working/models/technique_mapping.joblib'):
    if not os.path.exists(mapping_path):
        print("Mapping file not found.")
        return

    mapping = joblib.load(mapping_path)
    print("Technique Mapping:")
    for technique, code in mapping.items():
        print(f"{technique}: {code}")

In [ ]:
def clean_column_names(df):
    # Funzione per pulire i nomi delle colonne
    new_columns = []
    for col in df.columns:
        # Usa regex per separare le varietà
        varieties = re.findall(r'([a-z]+)_([a-z_]+)', col)
        if varieties:
            new_columns.append(f"{varieties[0][0]}_{varieties[0][1]}")
        else:
            new_columns.append(col)
    return new_columns


def prepare_comparison_data(simulated_data, olive_varieties):
    # Pulisci i nomi delle colonne
    df = simulated_data.copy()

    df.columns = clean_column_names(df)
    df = encode_techniques(df)

    all_varieties = olive_varieties['Varietà di Olive'].unique()
    varieties = [clean_column_name(variety) for variety in all_varieties]
    comparison_data = []

    for variety in varieties:
        olive_prod_col = next((col for col in df.columns if col.startswith(f'{variety}_') and col.endswith('_olive_prod')), None)
        oil_prod_col = next((col for col in df.columns if col.startswith(f'{variety}_') and col.endswith('_avg_oil_prod')), None)
        tech_col = next((col for col in df.columns if col.startswith(f'{variety}_') and col.endswith('_tech')), None)
        water_need_col = next((col for col in df.columns if col.startswith(f'{variety}_') and col.endswith('_water_need')), None)

        if olive_prod_col and oil_prod_col and tech_col and water_need_col:
            variety_data = df[[olive_prod_col, oil_prod_col, tech_col, water_need_col]]
            variety_data = variety_data[variety_data[tech_col] != 0]  # Esclude le righe dove la tecnica è 0

            if not variety_data.empty:
                avg_olive_prod = pd.to_numeric(variety_data[olive_prod_col], errors='coerce').mean()
                avg_oil_prod = pd.to_numeric(variety_data[oil_prod_col], errors='coerce').mean()
                avg_water_need = pd.to_numeric(variety_data[water_need_col], errors='coerce').mean()
                efficiency = avg_oil_prod / avg_olive_prod if avg_olive_prod > 0 else 0
                water_efficiency = avg_oil_prod / avg_water_need if avg_water_need > 0 else 0

                comparison_data.append({
                    'Variety': variety,
                    'Avg Olive Production (kg/ha)': avg_olive_prod,
                    'Avg Oil Production (L/ha)': avg_oil_prod,
                    'Avg Water Need (m³/ha)': avg_water_need,
                    'Oil Efficiency (L/kg)': efficiency,
                    'Water Efficiency (L oil/m³ water)': water_efficiency
                })

    return pd.DataFrame(comparison_data)


def plot_variety_comparison(comparison_data, metric):
    plt.figure(figsize=(12, 6))
    bars = plt.bar(comparison_data['Variety'], comparison_data[metric])
    plt.title(f'Comparison of {metric} across Olive Varieties')
    plt.xlabel('Variety')
    plt.ylabel(metric)
    plt.xticks(rotation=45, ha='right')

    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2., height,
                 f'{height:.2f}',
                 ha='center', va='bottom')

    plt.tight_layout()
    plt.show()
    save_plot(plt, f'variety_comparison_{metric.lower().replace(" ", "_").replace("/", "_").replace("(", "").replace(")", "")}')
    plt.close()


def plot_efficiency_vs_production(comparison_data):
    plt.figure(figsize=(10, 6))

    plt.scatter(comparison_data['Avg Olive Production (kg/ha)'],
                comparison_data['Oil Efficiency (L/kg)'],
                s=100)

    for i, row in comparison_data.iterrows():
        plt.annotate(row['Variety'],
                     (row['Avg Olive Production (kg/ha)'], row['Oil Efficiency (L/kg)']),
                     xytext=(5, 5), textcoords='offset points')

    plt.title('Oil Efficiency vs Olive Production by Variety')
    plt.xlabel('Average Olive Production (kg/ha)')
    plt.ylabel('Oil Efficiency (L oil / kg olives)')
    plt.tight_layout()
    save_plot(plt, 'efficiency_vs_production')
    plt.close()


def plot_water_efficiency_vs_production(comparison_data):
    plt.figure(figsize=(10, 6))

    plt.scatter(comparison_data['Avg Olive Production (kg/ha)'],
                comparison_data['Water Efficiency (L oil/m³ water)'],
                s=100)

    for i, row in comparison_data.iterrows():
        plt.annotate(row['Variety'],
                     (row['Avg Olive Production (kg/ha)'], row['Water Efficiency (L oil/m³ water)']),
                     xytext=(5, 5), textcoords='offset points')

    plt.title('Water Efficiency vs Olive Production by Variety')
    plt.xlabel('Average Olive Production (kg/ha)')
    plt.ylabel('Water Efficiency (L oil / m³ water)')
    plt.tight_layout()
    plt.show()
    save_plot(plt, 'water_efficiency_vs_production')
    plt.close()


def plot_water_need_vs_oil_production(comparison_data):
    plt.figure(figsize=(10, 6))

    plt.scatter(comparison_data['Avg Water Need (m³/ha)'],
                comparison_data['Avg Oil Production (L/ha)'],
                s=100)

    for i, row in comparison_data.iterrows():
        plt.annotate(row['Variety'],
                     (row['Avg Water Need (m³/ha)'], row['Avg Oil Production (L/ha)']),
                     xytext=(5, 5), textcoords='offset points')

    plt.title('Oil Production vs Water Need by Variety')
    plt.xlabel('Average Water Need (m³/ha)')
    plt.ylabel('Average Oil Production (L/ha)')
    plt.tight_layout()
    plt.show()
    save_plot(plt, 'water_need_vs_oil_production')
    plt.close()


def analyze_by_technique(simulated_data, olive_varieties):
    # Pulisci i nomi delle colonne
    df = simulated_data.copy()

    df.columns = clean_column_names(df)
    df = encode_techniques(df)
    all_varieties = olive_varieties['Varietà di Olive'].unique()
    varieties = [clean_column_name(variety) for variety in all_varieties]

    technique_data = []

    for variety in varieties:
        olive_prod_col = next((col for col in df.columns if col.startswith(f'{variety}_') and col.endswith('_olive_prod')), None)
        oil_prod_col = next((col for col in df.columns if col.startswith(f'{variety}_') and col.endswith('_avg_oil_prod')), None)
        tech_col = next((col for col in df.columns if col.startswith(f'{variety}_') and col.endswith('_tech')), None)
        water_need_col = next((col for col in df.columns if col.startswith(f'{variety}_') and col.endswith('_water_need')), None)

        if olive_prod_col and oil_prod_col and tech_col and water_need_col:
            variety_data = df[[olive_prod_col, oil_prod_col, tech_col, water_need_col]]
            variety_data = variety_data[variety_data[tech_col] != 0]

            if not variety_data.empty:
                for tech in variety_data[tech_col].unique():
                    tech_data = variety_data[variety_data[tech_col] == tech]

                    avg_olive_prod = pd.to_numeric(tech_data[olive_prod_col], errors='coerce').mean()
                    avg_oil_prod = pd.to_numeric(tech_data[oil_prod_col], errors='coerce').mean()
                    avg_water_need = pd.to_numeric(tech_data[water_need_col], errors='coerce').mean()

                    efficiency = avg_oil_prod / avg_olive_prod if avg_olive_prod > 0 else 0
                    water_efficiency = avg_oil_prod / avg_water_need if avg_water_need > 0 else 0

                    technique_data.append({
                        'Variety': variety,
                        'Technique': tech,
                        'Technique String': decode_single_technique(tech),
                        'Avg Olive Production (kg/ha)': avg_olive_prod,
                        'Avg Oil Production (L/ha)': avg_oil_prod,
                        'Avg Water Need (m³/ha)': avg_water_need,
                        'Oil Efficiency (L/kg)': efficiency,
                        'Water Efficiency (L oil/m³ water)': water_efficiency
                    })

    return pd.DataFrame(technique_data)

In [ ]:
def get_full_data(simulated_data, olive_varieties):
    # Assumiamo che simulated_data contenga già tutti i dati necessari
    # Includiamo solo le colonne rilevanti
    relevant_columns = ['year', 'temp_mean', 'precip_sum', 'solar_energy_sum', 'ha', 'zone', 'olive_prod']

    # Aggiungiamo le colonne specifiche per varietà
    all_varieties = olive_varieties['Varietà di Olive'].unique()
    varieties = [clean_column_name(variety) for variety in all_varieties]
    for variety in varieties:
        relevant_columns.extend([f'{variety}_olive_prod', f'{variety}_tech'])

    return simulated_data[relevant_columns].copy()


def analyze_correlations(full_data, variety):
    # Filtra i dati per la varietà specifica
    variety_data = full_data[[col for col in full_data.columns if not col.startswith('_') or col.startswith(f'{variety}_')]]

    # Rinomina le colonne per chiarezza
    variety_data = variety_data.rename(columns={
        f'{variety}_olive_prod': 'olive_production',
        f'{variety}_tech': 'technique'
    })

    # Matrice di correlazione
    plt.figure(figsize=(12, 10))
    corr_matrix = variety_data[['temp_mean', 'precip_sum', 'solar_energy_sum', 'olive_production']].corr()
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
    plt.title(f'Matrice di Correlazione - {variety}')
    plt.tight_layout()
    plt.show()
    save_plot(plt, f'correlation_matrix_{variety}')
    plt.close()

    # Scatter plots
    fig, axes = plt.subplots(2, 2, figsize=(20, 20))
    fig.suptitle(f'Relazione tra Fattori Meteorologici e Produzione di Olive - {variety}', fontsize=16)

    for ax, var in zip(axes.flat, ['temp_mean', 'precip_sum', 'solar_energy_sum', 'ha']):
        sns.scatterplot(data=variety_data, x=var, y='olive_production', hue='technique', ax=ax)
        ax.set_title(f'{var.capitalize()} vs Produzione Olive')
        ax.set_xlabel(var.capitalize())
        ax.set_ylabel('Produzione Olive (kg/ettaro)')

    plt.tight_layout()
    plt.show()
    save_plot(plt, f'meteorological_factors_{variety}')
    plt.close()

In [ ]:
def prepare_transformer_data(df, olive_varieties_df):
    # Crea una copia del DataFrame per evitare modifiche all'originale
    df = df.copy()

    # Ordina per zona e anno
    df = df.sort_values(['zone', 'year'])

    # Definisci le feature
    temporal_features = ['temp_mean', 'precip_sum', 'solar_energy_sum']
    static_features = ['ha']  # Feature statiche base
    target_features = ['olive_prod', 'min_oil_prod', 'max_oil_prod', 'avg_oil_prod', 'total_water_need']

    # Ottieni le varietà pulite
    all_varieties = olive_varieties_df['Varietà di Olive'].unique()
    varieties = [clean_column_name(variety) for variety in all_varieties]

    # Crea la struttura delle feature per ogni varietà
    variety_features = [
        'tech', 'pct', 'prod_t_ha', 'oil_prod_t_ha', 'oil_prod_l_ha',
        'min_yield_pct', 'max_yield_pct', 'min_oil_prod_l_ha', 'max_oil_prod_l_ha',
        'avg_oil_prod_l_ha', 'l_per_t', 'min_l_per_t', 'max_l_per_t', 'avg_l_per_t'
    ]

    # Prepara dizionari per le nuove colonne
    new_columns = {}

    # Prepara le feature per ogni varietà
    for variety in varieties:
        # Feature esistenti
        for feature in variety_features:
            col_name = f"{variety}_{feature}"
            if col_name in df.columns:
                if feature != 'tech':  # Non includere la colonna tech direttamente
                    static_features.append(col_name)

        # Feature binarie per le tecniche di coltivazione
        for technique in ['tradizionale', 'intensiva', 'superintensiva']:
            col_name = f"{variety}_{technique}"
            new_columns[col_name] = df[f"{variety}_tech"].notna() & (
                    df[f"{variety}_tech"].str.lower() == technique
            ).fillna(False)
            static_features.append(col_name)

    # Aggiungi tutte le nuove colonne in una volta sola
    new_df = pd.concat([df] + [pd.Series(v, name=k) for k, v in new_columns.items()], axis=1)

    # Ordiniamo per zona e anno per mantenere la continuità temporale
    df_sorted = new_df.sort_values(['zone', 'year'])

    # Definiamo la dimensione della finestra temporale
    window_size = 41

    # Liste per raccogliere i dati
    temporal_sequences = []
    static_features_list = []
    targets_list = []

    # Iteriamo per ogni zona
    for zone in df_sorted['zone'].unique():
        zone_data = df_sorted[df_sorted['zone'] == zone].reset_index(drop=True)

        if len(zone_data) >= window_size:  # Verifichiamo che ci siano abbastanza dati
            # Creiamo sequenze temporali scorrevoli
            for i in range(len(zone_data) - window_size + 1):
                # Sequenza temporale
                temporal_window = zone_data.iloc[i:i + window_size][temporal_features].values
                # Verifichiamo che non ci siano valori NaN
                if not np.isnan(temporal_window).any():
                    temporal_sequences.append(temporal_window)

                    # Feature statiche (prendiamo quelle dell'ultimo timestep della finestra)
                    static_features_list.append(zone_data.iloc[i + window_size - 1][static_features].values)

                    # Target (prendiamo quelli dell'ultimo timestep della finestra)
                    targets_list.append(zone_data.iloc[i + window_size - 1][target_features].values)

    # Convertiamo in array numpy
    X_temporal = np.array(temporal_sequences)
    X_static = np.array(static_features_list)
    y = np.array(targets_list)

    print(f"Dataset completo - Temporal: {X_temporal.shape}, Static: {X_static.shape}, Target: {y.shape}")

    # Split dei dati (usando indici casuali per una migliore distribuzione)
    indices = np.random.permutation(len(X_temporal))
    #train_idx = int(len(indices) * 0.7)
    #val_idx = int(len(indices) * 0.85)

    train_idx = int(len(indices) * 0.65)  # 65% training
    val_idx = int(len(indices) * 0.85)  # 20% validation
    # Il resto rimane 15% test

    # Oppure versione con 25% validation:
    #train_idx = int(len(indices) * 0.60)    # 60% training
    #val_idx = int(len(indices) * 0.85)      # 25% validation

    train_indices = indices[:train_idx]
    val_indices = indices[train_idx:val_idx]
    test_indices = indices[val_idx:]

    # Split dei dati
    X_temporal_train = X_temporal[train_indices]
    X_temporal_val = X_temporal[val_indices]
    X_temporal_test = X_temporal[test_indices]

    X_static_train = X_static[train_indices]
    X_static_val = X_static[val_indices]
    X_static_test = X_static[test_indices]

    y_train = y[train_indices]
    y_val = y[val_indices]
    y_test = y[test_indices]

    # Standardizzazione
    scaler_temporal = StandardScaler()
    scaler_static = StandardScaler()
    scaler_y = StandardScaler()

    # Standardizzazione dei dati temporali
    X_temporal_train = scaler_temporal.fit_transform(X_temporal_train.reshape(-1, len(temporal_features))).reshape(X_temporal_train.shape)
    X_temporal_val = scaler_temporal.transform(X_temporal_val.reshape(-1, len(temporal_features))).reshape(X_temporal_val.shape)
    X_temporal_test = scaler_temporal.transform(X_temporal_test.reshape(-1, len(temporal_features))).reshape(X_temporal_test.shape)

    # Standardizzazione dei dati statici
    X_static_train = scaler_static.fit_transform(X_static_train)
    X_static_val = scaler_static.transform(X_static_val)
    X_static_test = scaler_static.transform(X_static_test)

    # Standardizzazione dei target
    y_train = scaler_y.fit_transform(y_train)
    y_val = scaler_y.transform(y_val)
    y_test = scaler_y.transform(y_test)

    print("\nShape dopo lo split e standardizzazione:")
    print(f"Train - Temporal: {X_temporal_train.shape}, Static: {X_static_train.shape}, Target: {y_train.shape}")
    print(f"Val - Temporal: {X_temporal_val.shape}, Static: {X_static_val.shape}, Target: {y_val.shape}")
    print(f"Test - Temporal: {X_temporal_test.shape}, Static: {X_static_test.shape}, Target: {y_test.shape}")

    # Prepara i dizionari di input
    train_data = {'temporal': X_temporal_train, 'static': X_static_train}
    val_data = {'temporal': X_temporal_val, 'static': X_static_val}
    test_data = {'temporal': X_temporal_test, 'static': X_static_test}

    base_path = './kaggle/working/models/oil_transformer/'

    os.makedirs(base_path, exist_ok=True)

    joblib.dump(scaler_temporal, os.path.join(base_path, 'scaler_temporal.joblib'))
    joblib.dump(scaler_static, os.path.join(base_path, 'scaler_static.joblib'))
    joblib.dump(scaler_y, os.path.join(base_path, 'scaler_y.joblib'))

    return (train_data, y_train), (val_data, y_val), (test_data, y_test), (scaler_temporal, scaler_static, scaler_y)

In [ ]:
# Per denormalizzare e calcolare l'errore reale
def calculate_real_error(model, test_data, test_targets, scaler_y):
    # Fare predizioni
    predictions = model.predict(test_data)

    # Denormalizzare predizioni e target
    predictions_real = scaler_y.inverse_transform(predictions)
    targets_real = scaler_y.inverse_transform(test_targets)

    # Calcolare errore percentuale per ogni target
    percentage_errors = []
    absolute_errors = []

    for i in range(predictions_real.shape[1]):
        mae = np.mean(np.abs(predictions_real[:, i] - targets_real[:, i]))
        mape = np.mean(np.abs((predictions_real[:, i] - targets_real[:, i]) / targets_real[:, i])) * 100
        percentage_errors.append(mape)
        absolute_errors.append(mae)

    # Stampa risultati per ogni target
    target_names = ['olive_prod', 'min_oil_prod', 'max_oil_prod', 'avg_oil_prod', 'total_water_need']

    print("\nErrori per target:")
    print("-" * 50)
    for i, target in enumerate(target_names):
        print(f"{target}:")
        print(f"MAE assoluto: {absolute_errors[i]:.2f}")
        print(f"Errore percentuale medio: {percentage_errors[i]:.2f}%")
        print(f"Precisione: {100 - percentage_errors[i]:.2f}%")
        print("-" * 50)

    return percentage_errors, absolute_errors

In [ ]:
folder_path = './data/weather'
#raw_data = read_json_files(folder_path)
#weather_data = create_weather_dataset(raw_data)
#weather_data['datetime'] = pd.to_datetime(weather_data['datetime'], errors='coerce')
#weather_data['date'] = weather_data['datetime'].dt.date
#weather_data = weather_data.dropna(subset=['datetime'])
#weather_data['datetime'] = pd.to_datetime(weather_data['datetime'])
#weather_data['year'] = weather_data['datetime'].dt.year
#weather_data['month'] = weather_data['datetime'].dt.month
#weather_data['day'] = weather_data['datetime'].dt.day
#weather_data.head()

#weather_data.to_parquet('./data/weather_data.parquet')

In [ ]:
weather_data = pd.read_parquet('./kaggle/input/olive-oil/weather_data.parquet')

features = [
               'temp', 'tempmin', 'tempmax', 'humidity', 'cloudcover', 'windspeed', 'pressure', 'visibility',
               'hour_sin', 'hour_cos', 'month_sin', 'month_cos', 'day_of_year_sin', 'day_of_year_cos',
               'temp_humidity', 'temp_cloudcover', 'visibility_cloudcover', 'clear_sky_factor', 'day_length',
               'temp_1h_lag', 'cloudcover_1h_lag', 'humidity_1h_lag', 'temp_rolling_mean_6h',
               'cloudcover_rolling_mean_6h'
           ] + [col for col in weather_data.columns if 'season_' in col or 'time_period_' in col]


In [ ]:
training_params = {
    'epochs': 100,
    'batch_size': 32,
    'verbose': 1
}

X_scaled, scaler_X, y_scaled, scaler_y, data_after_2010 = prepare_solar_data(weather_data, features)

radiation_scalers = {
    'X': scaler_X,
    'y': scaler_y,
    'solar_params': solar_params_scaler
}

radiation_model, radiation_history = train_radiation_model(
    X_train_radiation,
    y_train_radiation,
    X_val_radiation,
    y_val_radiation,
    solar_params_train,
    solar_params_val,
    scalers=radiation_scalers,
    **training_params
)

In [ ]:
target_variables = ['solarradiation', 'solarenergy', 'uvindex']

# Salva tutto direttamente
save_models_and_scalers(
    models=models,
    scalers=scalers,  # Passiamo direttamente il dizionario degli scalers così com'è
    target_variables=target_variables
)

In [ ]:
data_after_2010 = weather_data[weather_data['year'] >= 2010].copy()
data_before_2010 = weather_data[weather_data['year'] < 2010].copy()
# Previsione delle variabili mancanti per data_before_2010
# Prepara data_before_2010
data_before_2010 = data_before_2010.sort_values('datetime')
data_before_2010.set_index('datetime', inplace=True)

data_after_2010 = data_after_2010.sort_values('datetime')
data_after_2010.set_index('datetime', inplace=True)

# Assicurati che le features non abbiano valori mancanti
data_before_2010[features] = data_before_2010[features].ffill()
data_before_2010[features] = data_before_2010[features].bfill()

In [ ]:
#models, scaler_X, scalers_y, target_variables = load_models_and_scalers()

# Effettua predizioni
predictions = predict_solar_variables(
    data_before_2010=data_before_2010,
    features=features,
    models=models,
    scalers=scalers,  # dizionario completo degli scalers
    target_variables=target_variables
)

# Crea dataset completo
weather_data_complete = create_complete_dataset(
    data_before_2010,
    data_after_2010,
    predictions
)

# Salva il risultato
weather_data_complete.reset_index(inplace=True)
weather_data_complete.to_parquet(
    './kaggle/working/data/weather_data_complete.parquet',
    index=False
)

## 2. Esplorazione dei Dati Meteo

In [ ]:
weather_data = pd.read_parquet('./kaggle/working/data/weather_data_complete.parquet')

In [ ]:
# Visualizzazione delle tendenze temporali
fig, axes = plt.subplots(6, 1, figsize=(15, 20))
weather_data.set_index('date')['temp'].plot(ax=axes[0], title='Temperatura Media Giornaliera')
weather_data.set_index('date')['humidity'].plot(ax=axes[1], title='Umidità Media Giornaliera')
weather_data.set_index('date')['solarradiation'].plot(ax=axes[2], title='Radiazione Solare Giornaliera')
weather_data.set_index('date')['solarenergy'].plot(ax=axes[3], title='Radiazione Solare Giornaliera')
weather_data.set_index('date')['uvindex'].plot(ax=axes[4], title='Precipitazioni Giornaliere')
weather_data.set_index('date')['precip'].plot(ax=axes[4], title='Precipitazioni Giornaliere')
plt.tight_layout()
plt.show()
save_plot(plt, 'weather_trends')
plt.close()

## 3. Simulazione dei Dati di Produzione Annuale

In [ ]:
olive_varieties = pd.read_csv('./kaggle/input/olive-oil/variety_olive_oil_production.csv')

olive_varieties = add_olive_water_consumption_correlation(olive_varieties)

olive_varieties.to_parquet("./kaggle/working/data/olive_varieties.parquet")

In [ ]:
olive_varieties = pd.read_parquet("./kaggle/working/data/olive_varieties.parquet")

weather_data = pd.read_parquet('./kaggle/working/data/weather_data_complete.parquet')

simulated_data = simulate_olive_production_parallel(weather_data, olive_varieties, 1000, random_state_value)

In [ ]:
# Visualizza il mapping delle tecniche
print_technique_mapping()

In [ ]:
simulated_data = pd.read_parquet("./kaggle/working/data/simulated_data.parquet")

# Esecuzione dell'analisi
comparison_data = prepare_comparison_data(simulated_data, olive_varieties)

# Genera i grafici
plot_variety_comparison(comparison_data, 'Avg Olive Production (kg/ha)')
plot_variety_comparison(comparison_data, 'Avg Oil Production (L/ha)')
plot_variety_comparison(comparison_data, 'Avg Water Need (m³/ha)')
plot_variety_comparison(comparison_data, 'Oil Efficiency (L/kg)')
plot_variety_comparison(comparison_data, 'Water Efficiency (L oil/m³ water)')
plot_efficiency_vs_production(comparison_data)
plot_water_efficiency_vs_production(comparison_data)
plot_water_need_vs_oil_production(comparison_data)

# Analisi per tecnica
technique_data = analyze_by_technique(simulated_data, olive_varieties)

print(technique_data)

# Stampa un sommario statistico
print("Comparison by Variety:")
print(comparison_data.set_index('Variety'))
print("\nBest Varieties by Water Efficiency:")
print(comparison_data.sort_values('Water Efficiency (L oil/m³ water)', ascending=False).head())

## 4. Analisi della Relazione tra Meteo e Produzione

In [ ]:
# Uso delle funzioni
full_data = get_full_data(simulated_data, olive_varieties)

# Assumiamo che 'selected_variety' sia definito altrove nel codice
# Per esempio:
selected_variety = 'nocellara_delletna'

analyze_correlations(full_data, selected_variety)

## 5. Preparazione del Modello di Machine Learning

## Divisione train/validation/test:


In [ ]:
simulated_data = pd.read_parquet("./kaggle/working/data/simulated_data.parquet")
olive_varieties = pd.read_parquet("./kaggle/working/data/olive_varieties.parquet")

(train_data, train_targets), (val_data, val_targets), (test_data, test_targets), scalers = prepare_transformer_data(simulated_data, olive_varieties)

scaler_temporal, scaler_static, scaler_y = scalers

print("Temporal data shape:", train_data['temporal'].shape)
print("Static data shape:", train_data['static'].shape)
print("Target shape:", train_targets.shape)

## OliveOilTransformer

In [ ]:
@keras.saving.register_keras_serializable()
class DataAugmentation(tf.keras.layers.Layer):
    """Custom layer per l'augmentation dei dati"""

    def __init__(self, noise_stddev=0.03, **kwargs):
        super().__init__(**kwargs)
        self.noise_stddev = noise_stddev

    def call(self, inputs, training=None):
        if training:
            return inputs + tf.random.normal(
                shape=tf.shape(inputs),
                mean=0.0,
                stddev=self.noise_stddev
            )
        return inputs

    def get_config(self):
        config = super().get_config()
        config.update({"noise_stddev": self.noise_stddev})
        return config


@keras.saving.register_keras_serializable()
class PositionalEncoding(tf.keras.layers.Layer):
    """Custom layer per l'encoding posizionale"""

    def __init__(self, d_model, **kwargs):
        super().__init__(**kwargs)
        self.d_model = d_model

    def build(self, input_shape):
        _, seq_length, _ = input_shape

        # Crea la matrice di encoding posizionale
        position = tf.range(seq_length, dtype=tf.float32)[:, tf.newaxis]
        div_term = tf.exp(
            tf.range(0, self.d_model, 2, dtype=tf.float32) *
            (-tf.math.log(10000.0) / self.d_model)
        )

        # Calcola sin e cos
        pos_encoding = tf.zeros((1, seq_length, self.d_model))
        pos_encoding_even = tf.sin(position * div_term)
        pos_encoding_odd = tf.cos(position * div_term)

        # Assegna i valori alle posizioni pari e dispari
        pos_encoding = tf.concat(
            [tf.expand_dims(pos_encoding_even, -1),
             tf.expand_dims(pos_encoding_odd, -1)],
            axis=-1
        )
        pos_encoding = tf.reshape(pos_encoding, (1, seq_length, -1))
        pos_encoding = pos_encoding[:, :, :self.d_model]

        # Salva l'encoding come peso non trainabile
        self.pos_encoding = self.add_weight(
            shape=(1, seq_length, self.d_model),
            initializer=tf.keras.initializers.Constant(pos_encoding),
            trainable=False,
            name='positional_encoding'
        )

        super().build(input_shape)

    def call(self, inputs):
        # Broadcast l'encoding posizionale sul batch
        batch_size = tf.shape(inputs)[0]
        pos_encoding_tiled = tf.tile(self.pos_encoding, [batch_size, 1, 1])
        return inputs + pos_encoding_tiled

    def get_config(self):
        config = super().get_config()
        config.update({"d_model": self.d_model})
        return config


@keras.saving.register_keras_serializable()
class WarmUpLearningRateSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    """Custom learning rate schedule with linear warmup and exponential decay."""

    def __init__(self, initial_learning_rate=1e-3, warmup_steps=500, decay_steps=5000):
        super().__init__()
        self.initial_learning_rate = initial_learning_rate
        self.warmup_steps = warmup_steps
        self.decay_steps = decay_steps

    def __call__(self, step):
        warmup_pct = tf.cast(step, tf.float32) / self.warmup_steps
        warmup_lr = self.initial_learning_rate * warmup_pct
        decay_factor = tf.pow(0.1, tf.cast(step, tf.float32) / self.decay_steps)
        decayed_lr = self.initial_learning_rate * decay_factor
        return tf.where(step < self.warmup_steps, warmup_lr, decayed_lr)

    def get_config(self):
        return {
            'initial_learning_rate': self.initial_learning_rate,
            'warmup_steps': self.warmup_steps,
            'decay_steps': self.decay_steps
        }


def create_olive_oil_transformer(temporal_shape, static_shape, num_outputs,
                                 d_model=128, num_heads=8, ff_dim=256,
                                 num_transformer_blocks=4, mlp_units=[256, 128, 64],
                                 dropout=0.2):
    """
    Crea un transformer per la predizione della produzione di olio d'oliva.
    """
    # Input layers
    temporal_input = tf.keras.layers.Input(shape=temporal_shape, name='temporal')
    static_input = tf.keras.layers.Input(shape=static_shape, name='static')

    # === TEMPORAL PATH ===
    x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(temporal_input)
    x = DataAugmentation()(x)

    # Temporal projection
    x = tf.keras.layers.Dense(
        d_model // 2,
        activation='gelu',
        kernel_regularizer=tf.keras.regularizers.l2(1e-5)
    )(x)
    x = tf.keras.layers.Dropout(dropout)(x)
    x = tf.keras.layers.Dense(
        d_model,
        activation='gelu',
        kernel_regularizer=tf.keras.regularizers.l2(1e-5)
    )(x)

    # Positional encoding
    x = PositionalEncoding(d_model)(x)

    # Transformer blocks
    skip_connection = x
    for _ in range(num_transformer_blocks):
        # Self-attention
        attention_output = tf.keras.layers.MultiHeadAttention(
            num_heads=num_heads,
            key_dim=d_model // num_heads,
            value_dim=d_model // num_heads
        )(x, x)
        attention_output = tf.keras.layers.Dropout(dropout)(attention_output)

        # Residual connection con pesi addestrabili
        residual_weights = tf.keras.layers.Dense(d_model, activation='sigmoid')(x)
        x = tf.keras.layers.Add()([x, residual_weights * attention_output])
        x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x)

        # Feed-forward network
        ffn = tf.keras.layers.Dense(ff_dim, activation="gelu")(x)
        ffn = tf.keras.layers.Dropout(dropout)(ffn)
        ffn = tf.keras.layers.Dense(d_model)(ffn)
        ffn = tf.keras.layers.Dropout(dropout)(ffn)

        # Second residual connection
        x = tf.keras.layers.Add()([x, ffn])
        x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x)

    # Add final skip connection
    x = tf.keras.layers.Add()([x, skip_connection])

    # Temporal pooling
    attention_pooled = tf.keras.layers.MultiHeadAttention(
        num_heads=num_heads,
        key_dim=d_model // 4
    )(x, x)
    attention_pooled = tf.keras.layers.GlobalAveragePooling1D()(attention_pooled)

    # Additional pooling operations
    avg_pooled = tf.keras.layers.GlobalAveragePooling1D()(x)
    max_pooled = tf.keras.layers.GlobalMaxPooling1D()(x)

    # Combine pooling results
    temporal_features = tf.keras.layers.Concatenate()(
        [attention_pooled, avg_pooled, max_pooled]
    )

    # === STATIC PATH ===
    static_features = tf.keras.layers.LayerNormalization(epsilon=1e-6)(static_input)
    for units in [256, 128, 64]:
        static_features = tf.keras.layers.Dense(
            units,
            activation='gelu',
            kernel_regularizer=tf.keras.regularizers.l2(1e-5)
        )(static_features)
        static_features = tf.keras.layers.Dropout(dropout)(static_features)

    # === FEATURE FUSION ===
    combined = tf.keras.layers.Concatenate()([temporal_features, static_features])

    # === MLP HEAD ===
    x = combined
    for units in mlp_units:
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dense(
            units,
            activation="gelu",
            kernel_regularizer=tf.keras.regularizers.l2(1e-5)
        )(x)
        x = tf.keras.layers.Dropout(dropout)(x)

    # Output layer
    outputs = tf.keras.layers.Dense(
        num_outputs,
        activation='linear',
        kernel_regularizer=tf.keras.regularizers.l2(1e-5)
    )(x)

    # Create model
    model = tf.keras.Model(
        inputs={'temporal': temporal_input, 'static': static_input},
        outputs=outputs,
        name='OilTransformer'
    )

    return model


def create_transformer_callbacks(target_names, val_data, val_targets):
    """
    Crea i callbacks per il training del modello.
    
    Parameters:
    -----------
    target_names : list
        Lista dei nomi dei target per il monitoraggio specifico
    val_data : dict
        Dati di validazione
    val_targets : array
        Target di validazione
    
    Returns:
    --------
    list
        Lista dei callbacks configurati
    """

    # Custom Metric per target specifici
    class TargetSpecificMetric(tf.keras.callbacks.Callback):
        def __init__(self, validation_data, target_names):
            super().__init__()
            self.validation_data = validation_data
            self.target_names = target_names

        def on_epoch_end(self, epoch, logs={}):
            x_val, y_val = self.validation_data
            y_pred = self.model.predict(x_val, verbose=0)

            for i, name in enumerate(self.target_names):
                mae = np.mean(np.abs(y_val[:, i] - y_pred[:, i]))
                logs[f'val_{name}_mae'] = mae

    # Crea le cartelle per i checkpoint e i log se non esistono
    os.makedirs('./kaggle/working/models/oil_transformer/checkpoints', exist_ok=True)
    os.makedirs('./kaggle/working/models/oil_transformer/logs', exist_ok=True)

    callbacks = [
        # Early Stopping
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=20,
            restore_best_weights=True,
            min_delta=0.0005,
            mode='min'
        ),

        # Model Checkpoint
        tf.keras.callbacks.ModelCheckpoint(
            filepath='./kaggle/working/models/oil_transformer/checkpoints/model_{epoch:02d}_{val_loss:.4f}.h5',
            monitor='val_loss',
            save_best_only=True,
            mode='min',
            save_weights_only=True
        ),

        # Metric per target specifici
        TargetSpecificMetric(
            validation_data=(val_data, val_targets),
            target_names=target_names
        ),

        # Reduce LR on Plateau
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=10,
            min_lr=1e-6,
            verbose=1
        ),

        # TensorBoard logging
        tf.keras.callbacks.TensorBoard(
            log_dir='./kaggle/working/models/oil_transformer/logs',
            histogram_freq=1,
            write_graph=True,
            update_freq='epoch'
        )
    ]

    return callbacks


def compile_model(model, learning_rate=1e-3):
    """
    Compila il modello con le impostazioni standard.
    """
    lr_schedule = WarmUpLearningRateSchedule(
        initial_learning_rate=learning_rate,
        warmup_steps=500,
        decay_steps=5000
    )

    model.compile(
        optimizer=tf.keras.optimizers.AdamW(
            learning_rate=lr_schedule,
            weight_decay=0.01
        ),
        loss=tf.keras.losses.Huber(),
        metrics=['mae']
    )

    return model


def setup_transformer_training(train_data, train_targets, val_data, val_targets):
    """
    Configura e prepara il transformer con dimensioni dinamiche basate sui dati.
    """
    # Estrai le shape dai dati
    temporal_shape = (train_data['temporal'].shape[1], train_data['temporal'].shape[2])
    static_shape = (train_data['static'].shape[1],)
    num_outputs = train_targets.shape[1]

    print(f"Shape rilevate:")
    print(f"- Temporal shape: {temporal_shape}")
    print(f"- Static shape: {static_shape}")
    print(f"- Numero di output: {num_outputs}")

    # Target names basati sul numero di output
    target_names = ['olive_prod', 'min_oil_prod', 'max_oil_prod', 'avg_oil_prod', 'total_water_need']

    # Assicurati che il numero di target names corrisponda al numero di output
    assert len(target_names) == num_outputs, \
        f"Il numero di target names ({len(target_names)}) non corrisponde al numero di output ({num_outputs})"

    # Crea il modello con le dimensioni rilevate
    model = create_olive_oil_transformer(
        temporal_shape=temporal_shape,
        static_shape=static_shape,
        num_outputs=num_outputs
    )

    # Compila il modello
    model = compile_model(model)

    # Crea i callbacks
    callbacks = create_transformer_callbacks(target_names, val_data, val_targets)

    return model, callbacks, target_names


def train_transformer(train_data, train_targets, val_data, val_targets, epochs=150, batch_size=64, save_name='final_model'):
    """
    Funzione principale per l'addestramento del transformer.
    """
    # Setup del modello
    model, callbacks, target_names = setup_transformer_training(
        train_data, train_targets, val_data, val_targets
    )

    # Mostra il summary del modello
    model.summary()
    os.makedirs(f"./kaggle/working/models/oil_transformer/", exist_ok=True)
    keras.utils.plot_model(model, f"./kaggle/working/models/oil_transformer/{save_name}.png", show_shapes=True)

    # Training
    history = model.fit(
        x=train_data,
        y=train_targets,
        validation_data=(val_data, val_targets),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=callbacks,
        verbose=1,
        shuffle=True
    )

    # Salva il modello finale
    save_path = f'./kaggle/working/models/oil_transformer/{save_name}.keras'
    model.save(save_path, save_format='keras')

    os.makedirs(f'./kaggle/working/models/oil_transformer/weights/', exist_ok=True)
    model.save_weights(f'./kaggle/working/models/oil_transformer/weights')
    print(f"\nModello salvato in: {save_path}")

    return model, history

## Model Training

In [ ]:
model, history = train_transformer(train_data, train_targets, val_data, val_targets)

In [ ]:
# Calcola gli errori reali
percentage_errors, absolute_errors = calculate_real_error(model, val_data, val_targets, scaler_y)

In [ ]:
def evaluate_model_performance(model, data, targets, set_name=""):
    """
    Valuta le performance del modello su un set di dati specifico.
    """
    predictions = model.predict(data, verbose=0)

    target_names = ['olive_prod', 'min_oil_prod', 'max_oil_prod', 'avg_oil_prod', 'total_water_need']
    metrics = {}

    for i, name in enumerate(target_names):
        mae = np.mean(np.abs(targets[:, i] - predictions[:, i]))
        mse = np.mean(np.square(targets[:, i] - predictions[:, i]))
        rmse = np.sqrt(mse)
        mape = np.mean(np.abs((targets[:, i] - predictions[:, i]) / (targets[:, i] + 1e-7))) * 100

        metrics[f"{name}_mae"] = mae
        metrics[f"{name}_rmse"] = rmse
        metrics[f"{name}_mape"] = mape

    if set_name:
        print(f"\nPerformance sul set {set_name}:")
        for metric, value in metrics.items():
            print(f"{metric}: {value:.4f}")

    return metrics


def retrain_model(base_model, train_data, train_targets,
                  val_data, val_targets,
                  test_data, test_targets,
                  epochs=50, batch_size=128):
    """
    Implementa il retraining del modello con i dati combinati.
    """
    print("Valutazione performance iniziali del modello...")
    initial_metrics = {
        'train': evaluate_model_performance(base_model, train_data, train_targets, "training"),
        'val': evaluate_model_performance(base_model, val_data, val_targets, "validazione"),
        'test': evaluate_model_performance(base_model, test_data, test_targets, "test")
    }

    # Combina i dati per il retraining
    combined_data = {
        'temporal': np.concatenate([train_data['temporal'], val_data['temporal'], test_data['temporal']]),
        'static': np.concatenate([train_data['static'], val_data['static'], test_data['static']])
    }
    combined_targets = np.concatenate([train_targets, val_targets, test_targets])

    # Crea una nuova suddivisione per la validazione
    indices = np.arange(len(combined_targets))
    np.random.shuffle(indices)

    split_idx = int(len(indices) * 0.9)
    train_idx, val_idx = indices[:split_idx], indices[split_idx:]

    # Prepara i dati per il retraining
    retrain_data = {k: v[train_idx] for k, v in combined_data.items()}
    retrain_targets = combined_targets[train_idx]
    retrain_val_data = {k: v[val_idx] for k, v in combined_data.items()}
    retrain_val_targets = combined_targets[val_idx]

    checkpoint_path = './kaggle/working/models/oil_transformer/retrain_checkpoints'
    os.makedirs(checkpoint_path, exist_ok=True)

    # Configura callbacks
    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True,
            min_delta=0.0001
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=5,
            min_lr=1e-6,
            verbose=1
        ),
        tf.keras.callbacks.ModelCheckpoint(
            filepath=os.path.join(checkpoint_path, 'model_{epoch:02d}_{val_loss:.4f}.keras'),
            monitor='val_loss',
            save_best_only=True,
            mode='min',
            save_weights_only=True
        )
    ]

    # Imposta learning rate per il fine-tuning
    optimizer = tf.keras.optimizers.AdamW(
        learning_rate=tf.keras.optimizers.schedules.ExponentialDecay(
            initial_learning_rate=1e-4,
            decay_steps=1000,
            decay_rate=0.9
        ),
        weight_decay=0.01
    )

    # Ricompila il modello con il nuovo optimizer
    base_model.compile(
        optimizer=optimizer,
        loss=tf.keras.losses.Huber(),
        metrics=['mae']
    )

    print("\nAvvio retraining...")
    history = base_model.fit(
        retrain_data,
        retrain_targets,
        validation_data=(retrain_val_data, retrain_val_targets),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=callbacks,
        verbose=1
    )

    print("\nValutazione performance finali...")
    final_metrics = {
        'train': evaluate_model_performance(base_model, train_data, train_targets, "training"),
        'val': evaluate_model_performance(base_model, val_data, val_targets, "validazione"),
        'test': evaluate_model_performance(base_model, test_data, test_targets, "test")
    }

    # Salva il modello finale
    save_path = './kaggle/working/models/oil_transformer/retrained_model.keras'
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    base_model.save(save_path, save_format='keras')
    print(f"\nModello riaddestrato salvato in: {save_path}")

    # Report miglioramenti
    print("\nMiglioramenti delle performance:")
    for dataset in ['train', 'val', 'test']:
        print(f"\nSet {dataset}:")
        for metric in initial_metrics[dataset].keys():
            initial = initial_metrics[dataset][metric]
            final = final_metrics[dataset][metric]
            improvement = ((initial - final) / initial) * 100
            print(f"{metric}: {improvement:.2f}% di miglioramento")

    return base_model, history, final_metrics


def start_retraining(model_path, train_data, train_targets,
                     val_data, val_targets,
                     test_data, test_targets,
                     epochs=50, batch_size=128):
    """
    Avvia il processo di retraining in modo sicuro.
    """
    try:
        print("Caricamento del modello...")
        base_model = tf.keras.models.load_model(model_path, compile=False)
        print("Modello caricato con successo!")

        return retrain_model(
            base_model=base_model,
            train_data=train_data,
            train_targets=train_targets,
            val_data=val_data,
            val_targets=val_targets,
            test_data=test_data,
            test_targets=test_targets,
            epochs=epochs,
            batch_size=batch_size
        )
    except Exception as e:
        print(f"Errore durante il retraining: {str(e)}")
        raise

In [ ]:
model_path = './kaggle/working/models/oil_transformer/final_model.keras'

retrained_model, retrain_history, final_metrics = start_retraining(
    model_path=model_path,
    train_data=train_data,
    train_targets=train_targets,
    val_data=val_data,
    val_targets=val_targets,
    test_data=test_data,
    test_targets=test_targets,
    epochs=50,
    batch_size=128
)

# Visualizza i risultati
visualize_retraining_results(retrain_history, initial_metrics, final_metrics)

## 8. Conclusioni e Prossimi Passi

In questo notebook, abbiamo:
1. Caricato e analizzato i dati meteorologici
2. Simulato la produzione annuale di olive basata sui dati meteo
3. Esplorato le relazioni tra variabili meteorologiche e produzione di olive
4. Creato e valutato un modello di machine learning per prevedere la produzione
5. Utilizzato ARIMA per fare previsioni meteo
6. Previsto la produzione di olive per il prossimo anno

Prossimi passi:
- Raccogliere dati reali sulla produzione di olive per sostituire i dati simulati
- Esplorare modelli più avanzati, come le reti neurali o i modelli di ensemble
- Incorporare altri fattori che potrebbero influenzare la produzione, come le pratiche agricole o l'età degli alberi
- Sviluppare una dashboard interattiva basata su questo modello